# Natural Language Processing Lab

This work was made on spanish for "FIng - UDELAR, Montevideo Uruguay" as a practice work for "Introducción al Procesamiento del Lenguaje Natural" (Introduction to NLP).

# Task 2

The objective of this task is to carry out various experiments to represent and classify texts. For this purpose, we will work with a corpus for sentiment analysis, created for the [TASS 2020](http://www.sepln.org/workshops/tass/) competition (IberLEF - SEPLN).



# Parte 1 - Carga y preprocesamiento del corpus

Para trabajar en este notebook deben cargar los tres archivos disponbiles en eva: train.csv, devel.csv y test.csv.

La aplicación de una etapa de preprocesamiento similar a la implementada en la tarea 1 es opcional. Es interesante hacer experimentos con y sin la etapa de preprocesamiento, de modo de comparar resultados (sobre el corpus de desarrollo, devel.csv) y definir si se incluye o no en la solución final.



In [ ]:
#Imports necesarios
import csv
import random
import re
import nltk
nltk.download('punkt')           #Para Stop words
from nltk.tokenize import word_tokenize
import numpy as np

In [ ]:
# Carga de los datasets

with open('train.csv', newline='', encoding="utf-8") as corpus_csv:
  reader = csv.reader(corpus_csv)
  next(reader) # Saltea el cabezal del archivo
  train_set = [x for x in reader]
  train_set_lexicos = [x for x in reader]

with open('devel.csv', newline='', encoding="utf-8") as corpus_csv2:
  reader2 = csv.reader(corpus_csv2)
  next(reader2) # Saltea el cabezal del archivo
  devel_set = [x for x in reader2]

with open('test.csv', newline='', encoding="utf-8") as corpus_csv3:
  reader3 = csv.reader(corpus_csv3)
  next(reader3) # Saltea el cabezal del archivo
  test_set = [x for x in reader3]

with open('lexico_pos_lemas_grande.csv', newline='', encoding="utf-8") as corpus_csv4:
  reader4 = csv.reader(corpus_csv4)
  next(reader4) # Saltea el cabezal del archivo
  pos_set = [x for x in reader4]

with open('lexico_neg_lemas_grande.csv', newline='', encoding="utf-8") as corpus_csv5:
  reader5 = csv.reader(corpus_csv5)
  next(reader5) # Saltea el cabezal del archivo
  neg_set = [x for x in reader5]

#Cargamos Stopwords
with open('stop_words_esp_anasent.csv', newline='', encoding="utf-8") as stop_words_csv:
  reader = csv.reader(stop_words_csv)
  next(reader) # Saltea el cabezal del archivo
  stop_words_set = [x[0] for x in reader]


#Definimos el corpus train_lexicos siendo este train y los léxicos proporcionados agregados como tweets:
for elem in neg_set:
  train_set_lexicos.insert(0,[0,elem[0],'N'])

for elem in pos_set:
  train_set_lexicos.insert(0,[0,elem[0],'P'])


# Elegir un tweet aleatorio para el corpus train e imprimirlo junto a su categoría
random_tweet = random.choice(train_set)
print(f"El tweet tiene id: {random_tweet[0]}")
print(f"El tweet es: {random_tweet[1]}")
print(f"y su categoría: {random_tweet[2]}")

Como se puede observar, al Corpus de entrenamiento decidimos agregarle en un nuevo Data Set: train_set léxicos, los lemas positivos y negativos brindados via EVA, con la correspondiente polaridad. A su vez crearemos un Data Set similar donde combinaremos palabras de los léxicos positivos y negativos, asignando la polaridad "NONE" a dichas combinaciones. Dicho Data Set sera: train_modificado, y sera utilizado para entrenar distintos modelos de clasificación impemnatdos en la Parte 3. Para intentar que train_modificado quede lo mas balanceado posible, primero determinamos con cuantos léxicos positivos y negativos contamos, y la proporción de las distintas polaridades en el Data Set de entrenamiento original:

In [ ]:
print("Cantidad de palabras que contiene el lexico positivo: ",len(pos_set))
print("Cantidad de palabras que contiene el lexico negativo: ",len(neg_set))
pos_train = 0
neg_train = 0
none_train = 0
for t in train_set:
  if t[2] =="P":
    pos_train +=1
  elif t[2] =="N":
    neg_train +=1
  else:
    none_train +=1

print("Cantidad de tweets positivos en train ",pos_train)
print("Cantidad de tweets negativos en train ",neg_train)
print("Cantidad de tweets neutros en train ",none_train) #suma = 8.314

Cantidad de palabras que contiene el lexico positivo:  3354
Cantidad de palabras que contiene el lexico negativo:  4796
Cantidad de tweets positivos en train  2967
Cantidad de tweets negativos en train  2639
Cantidad de tweets neutros en train  2708


En función de esto, agregaremos a train_modifiado, 3354 "tweets" positivos, 3354 "tweets" negativos y 3354 "tweets" neutros:

In [ ]:
train_modificado = train_set.copy()
for elem in range(0,3353):
  auxP = pos_set[elem]
  auxN = neg_set[elem]
  auxNone = auxP[0] + " " + auxN[0]
  train_modificado.insert(0,[0,auxNone,'NONE'])
  train_modificado.insert(0,[0,auxP[0],'P'])
  train_modificado.insert(0,[0,auxN[0],'N'])
print("Cantidad de tweets en train_modificado: ",len(train_modificado))

Cantidad de tweets en train_modificado:  18373


## Preprocesamiento de tweets

In [ ]:
# Preprocesamiento de los tweets

#############################################################################################
###########################  Código de preprocesamiento del LAB 1  ##########################
#############################################################################################

# reemplazar_URL(texto):
#Reemplaza las URL del texto recibido por parámetro por el string vacío.
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo
def reemplazar_URL(texto):
    texto2 = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# reemplazar_Usuario(texto):
#Reemplaza las menciones a usuarios del texto recibido por parámetro por el string vacío.
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo
def reemplazar_Usuario(texto):
    regex2 = r"@(\w+)"
    texto2 = re.sub(regex2,"", texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# reemplazar_Abreviaturas(texto):
#Remplazar abreviaturas comunes por el término original.
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo
def reemplazar_Abreviaturas(texto):
    texto2 = re.sub(r'[\s^][xX][qQ][\s$]',' porque ', texto)
    texto2 = re.sub(r'[\s^][pP](\s)*[qQ][\s$]',' porque ', texto2)
    texto2 = re.sub(r'porq',' porque ', texto2)
    texto2 = re.sub('[\s^][xX][\s$]',' por ', texto2)
    texto2 = re.sub('[\s^][qQ][\s$]',' que ', texto2)
    texto2 = re.sub('[\s^][kK][\s$]',' que ', texto2)
    texto2 = re.sub('[\s^][bB][nN][\s$]',' bien ', texto2)
    texto2 = re.sub('[\s^][tT][mM][bB][\s$]',' tambien ', texto2)
    # Consideramos que no corresponde reemplazar RT
    # como una abreviatura ya que puede cambiar el significado semántico de la oración,
    # y lo borramos directamente.
    texto2 = re.sub('[\s^][rR][tT][\s$]',' ', texto2)
    texto2 = re.sub('[\s^][aA][cC][eE][Ss][\s$]',' haces ', texto2)
    texto2 = re.sub('[\s^][bB][bB][\?*\s$]',' bebé ', texto2)
    texto2 = re.sub('[\s^][bB][bB][sS][\?*\s$]',' bebés ', texto2)
    texto2 = re.sub('[\s^][vV][sS][\s$]',' versus ', texto2)
    texto2 = re.sub('[\s^][cC][\s$]',' se ', texto2)
    texto2 = re.sub('[\s^]\+[\s$]',' mas ', texto2)
    texto2 = re.sub('[\s^][dD][\s$]',' de ', texto2)
    texto2 = re.sub('[\s^][dD][lL][\s$]',' del ', texto2)
    texto2 = re.sub('[\s^][tT][aA][\s$]',' está ', texto2)
    texto2 = re.sub('[\s^][pP][aA][\s$]',' para ', texto2)
    texto2 = re.sub('[\s^][pP][sS][\?*\.*\,*\s$]',' pues ', texto2)
    texto2 = re.sub('[\s^][mM][\s$]',' me ', texto2)
    texto2 = re.sub('[\s^][cC][sS][mM][\s$]',' insulto ', texto2)  #Cambiamos csm por insulto.
    texto2 = re.sub('[\s^][gG]ral[\s.$]',' general ', texto2)
    texto2 = re.sub('[\s^][dD][rR][.\s$]',' doctor ', texto2)
    texto2 = re.sub('[\s^][mM][gG][\s$]',' me gusta ', texto2)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# reemplazar_Risa(texto):
#Remplazar en este caso las variantes posibles de una risa (expresion frecuente) por el String "jajaja".
#Retorna además un identificador para controlar si hubo algún cambio en el tweet
#analizado, útil para realizar recorridas de testeo
def reemplazar_Risa(texto):
    er_risa = r'\b([aA]+[jJ]+[aA]+[aAjJ]*|[jJ]+[jaJA]+[jJ]+[jaJA]*|[aA]+[hH]+[aA]+[aAhH]*|[hH]+[haHA]+[hH]+[haHA]*|[oO]?[lL]+[oO]+[lL]+[oLOl]*|[aA]*[jaJA]+[jJ][jJAa]*|[eE]+[jJ]+[eE]+[eEjJ]*|[jJ]+[jeJE]+[jJ]+[jeJE]*|[eE]+[hH]+[eE]+[eEhH]*|[hH]+[heHE]+[hH]+[heHE]*|[eE]*[jeJE]+[jJ][jJeE]*)\b'
    texto2 = re.sub(er_risa,' jajaja ', texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# strip_accents(texto)
#Remueve los acentos del texto
def strip_accents(texto):
    texto2 = re.sub("á","a",texto)
    texto2 = re.sub("é","e",texto2)
    texto2 = re.sub("í","i",texto2)
    texto2 = re.sub("ó","o",texto2)
    texto2 = re.sub("ú","u",texto2)
    return texto2



# reemplazar_Hashtags
# reemplazamos hashtags del texto por el string "HASHTAG".
# Retorna además un identificador para controlar si hubo algún cambio en el tweet
# analizado, útil para realizar recorridas de testeo
def reemplazar_Hashtags(texto):
    er_hashtags = r'#'                      #Nos quedamos con lo que le seguia por si acaso.
    texto2 = re.sub(er_hashtags,'', texto)
    cambio = (texto2!=texto)
    return [texto2,cambio]



# reemplazar_repeticiones(texto)
# Reemplaza las repeticiones de 3 o más veces la misma letra por una sola en "texto"
def reemplazar_repeticiones(texto):
    texto = re.sub("aaa[a]*","a",texto)
    texto = re.sub("bbb[b]*","b",texto)
    texto = re.sub("ccc[c]*","c",texto)
    texto = re.sub("ddd[a]*","d",texto)
    texto = re.sub("eee[e]*","e",texto)
    texto = re.sub("fff[f]*","f",texto)
    texto = re.sub("ggg[g]*","g",texto)
    texto = re.sub("hhh[h]*","h",texto)
    texto = re.sub("iii[i]*","i",texto)
    texto = re.sub("jjj[j]*","j",texto)
    texto = re.sub("kkk[k]*","k",texto)
    texto = re.sub("lll[l]*","l",texto)
    texto = re.sub("mmm[m]*","m",texto)
    texto = re.sub("nnn[n]*","n",texto)
    texto = re.sub("ooo[o]*","o",texto)
    texto = re.sub("ppp[p]*","p",texto)
    texto = re.sub("qqq[q]*","q",texto)
    texto = re.sub("rrr[r]*","r",texto)
    texto = re.sub("sss[s]*","s",texto)
    texto = re.sub("ttt[t]*","t",texto)
    texto = re.sub("uuu[u]*","u",texto)
    texto = re.sub("vvv[v]*","v",texto)
    texto = re.sub("www[w]*","w",texto)  #Luego de Urls
    texto = re.sub("xxx[x]*","x",texto)
    texto = re.sub("yyy[y]*","y",texto)
    texto = re.sub("zzz[z]*","z",texto)
    return texto



# reemplazar_Groserias(texto)
# Reemplaza las groserías de "texto" por el string "GROSERIA"
# Retorna además un identificador para controlar si hubo algún cambio en el tweet
# analizado, útil para realizar recorridas de testeo
def reemplazar_Groserias(texto):
    regex1 = r'hij[oa]+[de]*[p]+[u]+[t]+[a]+|gilipollas|gilipolleces|bolud[oa]|mierda|mariconadas|estupid[oa]|estupide(z|ces)'
    regex2 = r'\sput[oa]\s|marica\s'
    texto2 = re.sub(regex1,' insulto ', texto)
    texto2 = re.sub(regex2,' insulto ', texto2)
    cambio = (texto2!=texto)
    return [texto2,cambio]



#Removemos 2 o mas espacios seguidos
def remover_espacios(texto):
  patron_espacios = r'\s+'
  texto = re.sub(patron_espacios, " ", texto)
  return texto

def remove_stop_words(texto):
  tokens_texto = word_tokenize(texto)
  sin_stop = [word for word in tokens_texto if not word in stop_words_set]
  texto_filtrado = (" ").join(sin_stop)
  return texto_filtrado

def alfanumericos(text):
    text=re.sub(r'[^a-zA-Z\s]',' ',text)
    return text

def remover_letras(texto):
  texto_limpio = re.sub(r'\b\w\b', '', texto)
  return texto_limpio

def eliminar_url(texto):
    patron_url = r'http'
    texto_limpio = re.sub(patron_url, '', texto)
    return texto_limpio

#procesar_tweet(tweet):
# Aplicamos todas las funciones anteriores para procesar un tweet
# y retornamos el resultado.
def procesar_tweet(tweet):
    if (tweet == ''):
      return ''
    contenido = ""
    contenido = tweet
    contenido = contenido.lower()
    contenido = strip_accents(contenido)
    cambio = True
    contenido,cambio = reemplazar_Usuario(contenido)
    contenido,cambio = reemplazar_Hashtags(contenido)
    contenido,cambio = reemplazar_URL(contenido)
    contenido = eliminar_url(contenido)
    contenido,cambio = reemplazar_Groserias(contenido)
    contenido,cambio = reemplazar_Abreviaturas(contenido)
    contenido = reemplazar_repeticiones(contenido)
    contenido,cambio = reemplazar_Risa(contenido)
    contenido = alfanumericos(contenido)
    contenido = remove_stop_words(contenido)
    contenido = remover_letras(contenido)
    contenido = remover_espacios(contenido)
    return contenido


Respecto al preprocesamiento que realizamos en el laboratorio 1, en esta ocasión eliminaremos las menciones y URLs.
Es decir, las sustituimos por el string vacío "". En cuanto a Hashtags, solo eliminaremos el símbolo "#", y no pondremos la palabra "HASHTAG" al reemplazar
A su vez, en esta ocasión nos resultará útil volver a pasar los tweets a minúsculas, y optaremos por eliminar las stopwords provistas en el EVA.
También eliminaremos los números y tíldes, ya que creemos que estos no aportan al objetivo, y reemplazaremos las distintas malas palabras
por la palabra "insulto", ya que esta se encuentra en la lista de lemas negativos, por lo que de este modo remarcará de mejor manera oraciones negativas. Además, en vez de preprocesar las risas y cambiarlas por el string 'jaja', decidimos cambiarlo por el string 'jajaja' ya que esta palabra se encuentra incluida en el léxico de palabras positivas y 'jaja' no. Como mencionamos, decidimos volver a eliminar los tildes para uniformizar los tweets ya que no siempre encontraremos las palabras con los tildes correctamente escritos en los tweets, como sucede habitualmente. Ademas, para utilizar los léxicos reconocer las palabras de los mismos la mayor cantidad de veces posible en los tweets. A su vez a diferencia de la tarea 1, no se hace un análisis sintáctico de los tweets, por lo que tildes y mayúsculas no tienen la misma relevancia.

# Parte 2 - Representación de los tweets

Para representar los tweets se pide que experimenten con modelos basados en Bag of Words (BoW) y con Word Embeddings.

Para los dos enfoques podrán elegir entre diferentes opciones:

**Bag of Words**

* BOW estándar: se recomienda trabajar con la clase [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) de sklearn, en particular, fit_transform y transform.
* BOW filtrando stop-words: tienen disponible en eva una lista de stop-words para el español, adaptada para análisis de sentimiento (no se filtran palabras relevantes para determinar la polaridad, como "no", "pero", etc.).
* BoW usando lemas: pueden usar herramientas de spacy.
* BOW seleccionando las features más relevantes: se recomienda usar la clase [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=select%20k%20best#sklearn.feature_selection.SelectKBest) y probar con diferentes valores de k (por ejemplo, 10, 50, 200, 1000).
* BOW combinado con TF-IDF: se recomienda usar la clase [TfidfVectorizer](https://https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

**Word Embeddings**

* A partir de los word embeddings, representar cada tweet como el vector promedio (mean vector) de los vectores de las palabras que lo componen.
* A partir de los word embeddings, representar cada tweet como la concatenación de los vectores de las palabras que lo componen (llevando el vector total a un largo fijo).

Se recomienda trabajar con alguna de las colecciones de word embeddings disponibles en https://github.com/dccuchile/spanish-word-embeddings. El repositorio incluye links a ejemplos y tutoriales.


Se pide que prueben al menos una opción basada en BoW y una basada en word embeddings.

Imports necesarios:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from scipy.sparse import csr_matrix, hstack
from collections import Counter
import pandas as pd
import math

## Representación de los Tweets utilizando Bag of Words:
  - En primer lugar, decidimos utilizar BOW combiando con TF-IDF, por lo que usamos las clases CountVectorizer y TfidfVectorizer, importadas anteriormente.

In [ ]:
# Representación de los tweets usando BoW

#BOW combinado con TF-IDF: utilizamos las clases CountVectorizer y TfidfVectorizer.
#preprocesamos los tweets del corpus de entrenamiento
def procesar_corpus(corpus):
  for j in range(0,len(corpus)):
    aux2 = corpus[j]
    aux2[1] = procesar_tweet(aux2[1])
    corpus[j] = aux2
  return corpus


train_set2 = train_modificado.copy()                          #Para mantener los data sets intactos
devel_set2 = devel_set.copy()
test_set_2  = test_set.copy()

Train_procesado = procesar_corpus(train_set2)
X_trainP = [''.join(words[1]) for words in Train_procesado]  #Tweets de Entrenamiento preprocesados

Devel_procesado = procesar_corpus(devel_set2)
X_develP = [''.join(words[1]) for words in Devel_procesado]  #Tweets de Desarrollo preprocesados

Test_procesado = procesar_corpus(test_set_2)
X_testP = [''.join(words[1]) for words in Test_procesado]    #Tweets de Test preprocesados

y_train_mod = [label[2] for label in train_set2]             #Etiquetas de Entrenamiento
y_devel = [label[2] for label in devel_set2]                  #Etiquetas de Desarrollo
y_test = [label[2] for label in test_set_2]                  #Etiquetas de Test

#min_df=1 establece que una palabra debe aparecer como minimo en uno de los tweets para ser considerada en el vocabulario.
vectorizer = TfidfVectorizer(min_df = 1)

X_train_bowP = vectorizer.fit_transform(X_trainP)            #Representamos los datos de entrenamiento en forma de BOW con esquema TF-IDF.
X_test_bowP = vectorizer.transform(X_testP)                  #Representamos los datos de test en forma de BOW con esquema TF-IDF.
X_devel_bowP = vectorizer.transform(X_develP)                #Representamos los datos de desarrollo en forma de BOW con esquema TF-IDF.

print(X_train_bowP.shape)
print(X_test_bowP.shape)
print(X_devel_bowP.shape)

(18373, 21771)
(1884, 21771)
(1132, 21771)


En las salidas anteriores, vemos como al representar los tweets de los distintos corpus mediante el enfoque de BOW previamente mencionado, queda determinado un vocabulario de 21771 palabras representativas, indicado por el segundo valor de las duplas. Estas 21771 palabras hacen referencia a la unión de las palabras que aparecen en los distintos tweets del corpus de entrenamiento,y los lemas positivos y negativos brindados via EVA. Por otro lado el primer valor representa la cantidad de tweets presentes en cada corpus, donde cada tweet tiene su correspondiente BOW.

Veamos para un tweet random del corpus:

In [ ]:
random_tweet = random.choice(train_set)
print(f"El tweet tiene id: {random_tweet[0]}")
print(f"El tweet es: {random_tweet[1]}")
print(f"Su categoría: {random_tweet[2]}")

tweet_procesado = procesar_tweet(random_tweet[1])
Aux = [tweet_procesado]
print(f"Luego de procesarlo: {tweet_procesado}")

vectorizer = TfidfVectorizer(min_df = 1)
X_train_bowPAux = vectorizer.fit_transform(X_trainP)         #con tweets procesados
tweet_bow = vectorizer.transform(Aux)
print("Representacion en BOW del tweet:")
print(tweet_bow)

El tweet tiene id: 769980058761068547
El tweet es: odisea libros favoritos shakespeare ejemplo encanta hamlet
Su categoría: P
Luego de procesarlo: odisea libros favoritos shakespeare ejemplo encanta hamlet
Representacion en BOW del tweet:
  (0, 19160)	0.42615975578703486
  (0, 14942)	0.42615975578703486
  (0, 12737)	0.34738011670321906
  (0, 10415)	0.42615975578703486
  (0, 9104)	0.35842239580637375
  (0, 7683)	0.30223542075553095
  (0, 7396)	0.3386410766386862


Como se puede ver, para el tweet aleatorio tendremos una cantidad de filas en la representación igual a la cantidad de palabras que cuenta el tweet luego de preprocesarlo. El segundo elemento de la dupla indica el índice de la palabra respecto al inixado que realiza la clase TfidfVectorizer, y a la derecha el correspondiente valor que esta también genera tomando en cuenta el esquema TF-IDF.

- En segundo lugar, optamos por BOWs donde seleccionamos las features mas relevantes, utilizando la clase SelectKBest. En particular se utilizaron los valores k = 250, k = 500 y k = 1000.

In [ ]:
def seleccionarK(Entrada,Salida,Ka):
  vectorizer = CountVectorizer()
  bag_of_words = vectorizer.fit_transform(Entrada)
  # Seleccionamos Ka características mas relevantes con SelectKBest
  k = Ka  # Número de características a seleccionar
  selector = SelectKBest(score_func=chi2, k=k)  # Utilizamos chi2 como función de puntuación

  selected_features = selector.fit_transform(bag_of_words, Salida)

  # Obtenemos el vocabulario y las características seleccionadas
  vocab = vectorizer.get_feature_names_out()
  selected_feature_indices = selector.get_support(indices=True)
  selected_vocab = [vocab[i] for i in selected_feature_indices]
  return selected_vocab

def representar_con_k(Entrada,Vocabulario_k):
  vectorizer = CountVectorizer(vocabulary=Vocabulario_k)
  bag_of_words = vectorizer.transform(Entrada)
  return bag_of_words

Por ejemplo veamos las palabras seleccionadas para el conjunto de Entrenamiento pre-procesado y con léxico negativo y positivo agregados al mismo, con k = 10.

In [ ]:
vocabularioAux = seleccionarK(X_trainP,y_train_mod,10)
for t in vocabularioAux:
  print(t)

deficit
enhorabuena
felicidades
feliz
gracias
gran
insulto
muy
no
portada


Ahora declararemos funciones que nos seran de utilidad a a la hora de requerir la representacion en BOW de los distintos Data Sets. Las mismas, son utilizadas de modo que a cada BOW que representa un tweet, se le agregara a dicha representación dos valores mas al final de este, de modo que en la anteúltima coordenada tendremos el numero de palabras en el tweet que se encuentran presentes en el léxico de palabras positivas. Del mismo modo la última coordenada del vector tendrá la cantidad de palabras del tweet que se encuentran en el léxico de palabras negativas.

In [ ]:
negativas = []
for elem in neg_set:
  negativas.insert(0,elem[0])

positivas = []
for elem in pos_set:
  positivas.insert(0,elem[0])


def contar_lemas(texto):
  tokens = word_tokenize(texto)
  pos = [pal[1] for pal in positivas]
  neg = [pal2[1] for pal2 in negativas]
  suma_pos = 0
  suma_neg = 0
  for tok in tokens:
    if tok in pos:
      suma_pos += 1
    if tok in neg:
      suma_neg +=1
  return [suma_pos,suma_neg]


def lemas_Conjunto(X):
  col = []
  for x in X:
    dos_valores = contar_lemas(x)
    col.append(dos_valores)
  return col

def agregar_a_bow(BOWS, Extras):
  j = 0
  for lista in BOWS:    #Largo de ambos es igual
    aux = csr_matrix([Extras[j]])
    matriz_combinada = hstack([lista, aux])
    lista = matriz_combinada
    j +=1
  return BOWS

extras = lemas_Conjunto(X_trainP)
extras2 = lemas_Conjunto(X_develP)

## Representación de los tweets utilizando Word Embeddings:
  Decidimos representar los tweets mediante word embeddings siguiendo la sugerencia de tomar el vector promedio y la concatenación de los word embeddings de las palabras de los tweets luego de ser preprocesados.\
  Para cargar el archivo de word embeddings utilizamos Google Drive, cargamos en archivo dentro de una carpeta llamada 'IPLN' en el directorio principal de nuestro drive y dentro colocamos el archivo que se puede obtener desde: https://fasttext.cc/docs/en/crawl-vectors.html en la sección 'Spanish', descargando el archivo .text. Este contiene 2 millones de WE.

In [ ]:
from google.colab import drive
from gensim.models.keyedvectors import KeyedVectors



#importar_vectores(limite_vectores): carga el archivo de embeddings tomando limite_vectores como limite.
def importar_vectores(limite_vectores):
  #Cargamos el archivo .vec con los vectores de palabras desde Drive:
  drive.mount('/content/drive')
  wordvectors_file_vec = '/content/drive/MyDrive/IPLN/cc.es.300.vec'
  wordvectors_col = KeyedVectors.load_word2vec_format(wordvectors_file_vec,limit=limite_vectores)
  vocabulario_size = wordvectors_col.vectors.shape[0]
  print('la colección importada tiene ' + str(vocabulario_size) + ' vectores cargados')
  return wordvectors_col



#cargar_lexico(lexico): toma un léxico y carga todos sus elementos en una lista
def cargar_lexico(lexico):
  ret = []
  for elem in lexico:
    ret.append(procesar_tweet(elem[0]))
  return ret
lexico_negativas = cargar_lexico(neg_set.copy())
lexico_positivas = cargar_lexico(pos_set.copy())

#importar_vectores(limite_vectores): carga el archivo de embeddings tomando limite_vectores como limite.
def importar_vectores(limite_vectores):
  #Cargamos el archivo .vec con los vectores de palabras desde Drive:
  drive.mount('/content/drive')
  wordvectors_file_vec = '/content/drive/MyDrive/IPLN/cc.es.300.vec'
  wordvectors_col = KeyedVectors.load_word2vec_format(wordvectors_file_vec,limit=limite_vectores)
  vocabulario_size = wordvectors_col.vectors.shape[0]
  print('la colección importada tiene ' + str(vocabulario_size) + ' vectores cargados')
  return wordvectors_col



#cargar_top_embeddings(limite_vectores,cantidad_top_palabras,corpus):
#Obtiene los 'cantidad_top_palabras' embeddings más comunes de 'corpus' y los almacena en un diccionario
#de tipo palabra -> embedding, si es que su embedding está definido en la coleccion importada 'col_vectores_importados'.
#Si cantidad_top_palabras es 0, devuelve todas las palabras con embedding del corpus
#NOTA: El corpus ya debe estar procesado.
def cargar_top_embeddings(col_vectores_importados,cantidad_top_palabras,corpus,incluir_lexicos):

  #Copiamos el corpus para procesarlo:
  corpus_carga = corpus.copy()
  X_carga = [''.join(words[1]) for words in corpus_carga]

  #Separamos en una lista todas las palabras del corpus:
  counter_palabras = Counter([words for tweets in X_carga for words in tweets.split()])
  df = pd.DataFrame()
  df['key'] = counter_palabras.keys()
  df['value'] = counter_palabras.values()
  df.sort_values(by='value', ascending=False, inplace=True)

  top_n_words = []
  #Tomamos las 18.000 palabras mas comunes:
  if (cantidad_top_palabras):
    top_n_words = list(df[:cantidad_top_palabras].key.values)
  else:
    top_n_words =  list(df.key.values)

  print('Las palabras mas comunes del corpus son: ' + str(top_n_words[:10]))

  #Obtenemos la colección de 'cantidad_top_palabras' vectores:
  coleccion_vectores_we = {}
  for word in top_n_words:
    if word in col_vectores_importados.key_to_index:
      if(incluir_lexicos):
        negativa=0
        positiva=0
        if word in lexico_negativas:
          negativa=1
        if word in lexico_positivas:
          positiva=1
        aux = col_vectores_importados.get_vector(word)
        v_lista = aux.tolist()
        #Agregamos dos ceros para generar atributos con los léxicos más adelante
        v_lista.append(negativa)
        v_lista.append(positiva)
        coleccion_vectores_we[word] = np.array(v_lista)
      else:
        aux = col_vectores_importados.get_vector(word)
        coleccion_vectores_we[word] = np.array(aux)
  print('Nuestra colección tiene ' + str(len(coleccion_vectores_we)) + ' vectores cargados')
  return coleccion_vectores_we

Como se puede apreciar con estos ejemplos, se importa la colección de word embeddings y verificamos que se tiene la información semántica.

In [ ]:
#   ####################################################  #
#   Ejemplos de representaciones utilizando wordvectors:  #
#   ####################################################  #


limite_vectores_importados = 50000
top_palabras_tweets = 18000
wordvectors = importar_vectores(limite_vectores_importados)

#Encontrando la palabra menos relacionada:
ej1 = wordvectors.doesnt_match(['sol','luna','almuerzo','estrellas'])
ej2 = wordvectors.doesnt_match(['blanco','azul','rojo','chile'])
print('\nEjemplo palabra menos relacionada: [sol,luna,almuerzo,estrellas]: ' + str(ej1))
print('\nEjemplo palabra menos relacionada: [blanco,azul,rojo,chile]: ' + str(ej2))

#Palabras más similares:

ej3 = wordvectors.most_similar_cosmul(positive=['fruta','amarillo'],negative=['color'])

print("\nTop 2 mas similares a 'fruta, amarillo' y diferentes a 'color': " + str([par[0] for par in ej3[:2]]))

ej4 = wordvectors.most_similar_cosmul(positive=['comida','noche'])

print("\nMás similar a 'comida, noche': " + str([par[0] for par in ej4[:1]]))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
la colección importada tiene 50000 vectores cargados

Ejemplo palabra menos relacionada: [sol,luna,almuerzo,estrellas]: almuerzo

Ejemplo palabra menos relacionada: [blanco,azul,rojo,chile]: chile

Top 2 mas similares a 'fruta, amarillo' y diferentes a 'color': ['piña', 'plátano']

Más similar a 'comida, noche': ['cena']


A modo de ejemplo, imprimimos un vector de embeddings para una palabra del español:

In [ ]:
print(wordvectors.get_vector('hola'))

[-5.670e-02  5.340e-02 -6.130e-02 -2.440e-01 -1.366e-01  5.670e-02
  5.910e-02 -1.810e-02 -9.960e-02 -1.206e-01 -5.050e-02  7.950e-02
  1.053e-01  6.820e-02  1.325e-01  3.310e-02 -4.620e-02  9.190e-02
 -1.750e-02  7.590e-02  5.230e-02 -5.350e-02  1.290e-02  1.149e-01
 -4.020e-02  3.160e-02 -1.514e-01  4.190e-02 -6.790e-02  2.320e-02
  3.070e-02  8.190e-02 -1.450e-02 -1.122e-01 -5.680e-02  3.400e-02
 -1.000e-03 -9.070e-02 -1.680e-02  1.009e-01  1.248e-01 -1.486e-01
 -6.620e-02 -5.230e-02 -2.742e-01  1.379e-01  4.000e-03  9.410e-02
 -3.590e-02  1.095e-01  6.020e-02  2.642e-01  6.180e-02  3.600e-03
 -2.710e-02  1.617e-01  6.400e-02 -5.940e-02  2.050e-02 -1.510e-02
  9.700e-03  2.760e-02 -1.165e-01  5.110e-02 -4.890e-02 -1.990e-02
 -2.530e-02  4.650e-02 -3.460e-02  2.451e-01 -1.217e-01  3.800e-02
  9.400e-03 -4.520e-02  4.680e-02  3.540e-02  2.700e-02 -1.145e-01
 -1.400e-03  1.509e-01 -6.750e-02  1.307e-01 -1.050e-01  5.300e-02
  1.350e-01 -3.120e-02  1.148e-01  1.210e-02  3.870e-02 -2.524

Importamos la colección de word embeddings y a continuación definimos nuestra propia colección tomando las palabras mas frecuentes de los tweets:

In [ ]:
limite_vectores_importados = 2000000
wordvectors = importar_vectores(limite_vectores_importados)

Creamos las colecciones a utilizar:

In [ ]:
coleccion_vectores_we =  cargar_top_embeddings(wordvectors,0,Train_procesado,False)
Cool = cargar_top_embeddings(wordvectors,0,Train_procesado,True)

In [ ]:
#Experimentacion con tweets:
#A partir de los word embeddings, representar cada tweet como el vector promedio
#(mean vector) de los vectores de las palabras que lo componen.

#splitted_tweet(tweet) -> recibe tweet "crudo", obtenido directamente de un corpus,
#lo preprocesa con la parte 1 y retorna el resultado de dividirlo en palabras con split().
def splitted_tweet(tweet):
  tweet_procesado = procesar_tweet(tweet)
  return tweet_procesado.split()


#obtener_vector_promedio(tweet_split) -> Recibe una lista de palabras en "tweet_split" y retorna en vector_promedio el vector promedio de la representacion con word
#embeddings de cada palabra del tweet y en cant_palabras, la cantidad de palabras del tweet.
def obtener_vector_promedio(tweet_split):
    vector_promedio = np.zeros(302)
    ret = np.zeros(302)
    palabras_negativas = 0
    palabras_positivas = 0
    cant_palabras = 0
    for palabra_tweet in tweet_split:
      if palabra_tweet in lexico_negativas:
          palabras_negativas+=1
      if palabra_tweet in lexico_positivas:
          palabras_positivas+=1
    vectores_tweet = []
    vectores_tweet2 = [coleccion_vectores_we[palabra] for palabra in tweet_split if palabra in coleccion_vectores_we]
    for v_tweet in vectores_tweet2:
      aux = v_tweet.tolist()
      aux.append(0)
      aux.append(0)
      vectores_tweet.append(np.array(aux))
    if vectores_tweet:
      matriz_palabras = np.array(vectores_tweet)
      vector_promedio = np.mean(matriz_palabras, axis=0)
      v_lista = np.zeros(302)
      v_lista[300] = palabras_negativas
      v_lista[301] = palabras_positivas
      ret = vector_promedio + np.array(v_lista)
    return ret,cant_palabras

#promedio_de_palabras(corpus) -> obtiene el promedio de las palabras del corpus corpus
#luego de este ser procesado
# NOTA: pasar un corpus copiado como parámetro.
def promedio_de_palabras(corpus):
  corpus_procesado = procesar_corpus(corpus)
  corpus_tweets = [x[1] for x in corpus_procesado]
  for tweet in corpus_tweets:
    t_split = tweet.split()
    for palabra in t_split:
      if(not (palabra in coleccion_vectores_we)):
        t_split.remove(palabra)
    tweet = (''.join(elem) for elem in t_split)
  cantidad_tweets = len(corpus_tweets)
  counter = [words for tweets in X_trainP for words in tweets.split()]
  return len(counter) / (cantidad_tweets)

#Definimos el largo máximo de la representación de las palabras

largo_maximo_en_palabras = math.trunc(promedio_de_palabras(train_set.copy()))
print('Largo promedio de las palabras de train set entrenado: ' + str(largo_maximo_en_palabras))

#obtener_vector_concatenacion(tweet_split): tweet split es una lista de strings de palabras, obtiene
#la representacion en concatenacion de los word embeddings de esa lista.
def obtener_vector_concatenacion(tweet_split):
  vector_concatenado = np.zeros(largo_maximo_en_palabras*300+2,dtype=np.float64)
  palabras_negativas = 0
  palabras_positivas = 0
  cant_palabras = 0
  vectores_tweet = []
  vectores_tweet2 = [coleccion_vectores_we[palabra] for palabra in tweet_split if palabra in coleccion_vectores_we]
  for v_tweet in vectores_tweet2:
    aux = v_tweet.tolist()
    aux.append(0)
    aux.append(0)
    vectores_tweet.append(np.array(aux))
  if(vectores_tweet):
    vector_count = np.array(vectores_tweet[0])
    vectores_tweet.pop(0)
    for vector_tweet in vectores_tweet:
      vector_count = np.concatenate((vector_count, np.array(vector_tweet)))
    vector_concatenado = vector_count
    diferencia = largo_maximo_en_palabras*300+2-vector_concatenado.shape[0]
    if diferencia > 0:
      vector_concatenado = np.pad(vector_concatenado, (0, largo_maximo_en_palabras*300+2-vector_concatenado.shape[0]), 'constant')
    else:
      vector_concatenado = vector_concatenado[:(largo_maximo_en_palabras*300+2)]
  for palabra_tweet in tweet_split:
      cant_palabras +=1
      if palabra_tweet in lexico_negativas:
          palabras_negativas+=1
      if palabra_tweet in lexico_positivas:
          palabras_positivas+=1
  vector_concatenado[largo_maximo_en_palabras*300] = palabras_negativas
  vector_concatenado[largo_maximo_en_palabras*300+1] = palabras_positivas
  return vector_concatenado



#word_embedding_promedio_corpus(corpus) Toma "corpus" el cual es una lista cargada desde la parte 1
#de la forma [[id,tweet,etiqueta],[id2,tweet2,etiqueta2], ... ]
#y devuelve el mismo corpus
#[[id,tweet_we,etiqueta],[id2,tweet2_we,etiqueta2], ... ] pero los elementos tweet_we son las representaciones
#en word embedding tomando el promedio de las palabras reconocidas del tweet
def word_embedding_promedio_corpus(corpus):
  corpus_emb = corpus.copy()
  we_corpus = []
  if(len(corpus_emb[0]) == 3):
    for elemento in corpus_emb:
      tweet = elemento[1]
      id = elemento[0]
      polaridad = elemento[2]
      if(not isinstance(tweet, list)):
        tweet_split = splitted_tweet(tweet)
        v_prom,n_palabras = obtener_vector_promedio(tweet_split)
        we_corpus.append([id,v_prom,polaridad])
      else:
        print(tweet)
  else:
    for elemento in corpus_emb:
      tweet = elemento[0]
      if(not isinstance(tweet, list)):
        tweet_split = splitted_tweet(tweet)
        v_prom,n_palabras = obtener_vector_promedio(tweet_split)
        we_corpus.append([v_prom])
      else:
        print(tweet)
  return we_corpus


#word_embedding_concatenacion_corpus(corpus) Toma "corpus" el cual es una lista cargada desde la parte 1
#de la forma [[id,tweet,etiqueta],[id2,tweet2,etiqueta2], ... ]
#y devuelve el mismo corpus
#[[id,tweet_we,etiqueta],[id2,tweet2_we,etiqueta2], ... ] pero los elementos tweet_we son las representaciones
#en word embedding tomando la concatenacion de las palabras reconocidas del tweet
def word_embedding_concatenacion_corpus(corpus):
  corpus_emb = corpus.copy()
  we_corpus = []
  for elemento in corpus_emb:
    tweet = elemento[1]
    id = elemento[0]
    polaridad = elemento[2]
    if(not isinstance(tweet, list)):
      tweet_split = splitted_tweet(tweet)
      v_prom = obtener_vector_concatenacion(tweet_split)
      we_corpus.append([id,v_prom,polaridad])
    else:
      print(tweet)
  return we_corpus


Procesamos los corpus y cargamos los datos a utilizar en las siguientes secciones:

In [ ]:
######################################################
################ Carga de datos para WE: #############
######################################################


##################### PROMEDIO ########################################

#Cargamos los corpus:
devel_set_we_promedio = []
train_set_we_promedio = []
test_set_we_promedio = []
train_set_lexicos_we_promedio = []

devel_set_we_promedio  = word_embedding_promedio_corpus(devel_set)
train_set_we_promedio  = word_embedding_promedio_corpus(train_set)
train_set_lexicos_we_promedio  = word_embedding_promedio_corpus(train_set_lexicos)
test_set_we_promedio   = word_embedding_promedio_corpus(test_set)


#Obtenemos por separado representaciones de los tweets:

X_train_lexicos_we_promedio = np.array([x[1] for x in train_set_lexicos_we_promedio])       # tweets train + léxicos
X_devel_we_promedio = np.array([x[1] for x in devel_set_we_promedio])                       # tweets devel
X_test_we_promedio = np.array([x[1] for x in test_set_we_promedio])                         # tweets test
X_train_we_promedio = np.array([x[1] for x in train_set_we_promedio])                       # tweets train


################################### CONCATENACION ##########################################

#Cargamos los corpus:
devel_set_we_concatenacion = []
train_set_we_concatenacion = []
test_set_we_concatenacion = []
train_set_lexicos_we_concatenacion = []

devel_set_we_concatenacion   = word_embedding_concatenacion_corpus(devel_set)
train_set_we_concatenacion  = word_embedding_concatenacion_corpus(train_set)
test_set_we_concatenacion   = word_embedding_concatenacion_corpus(test_set)
train_set_lexicos_we_concatenacion  = word_embedding_concatenacion_corpus(train_set_lexicos)

X_train_lexicos_we_concatenacion = np.array([x[1] for x in train_set_lexicos_we_concatenacion])     # tweets train + léxicos
X_devel_we_concatenacion = np.array([x[1] for x in devel_set_we_concatenacion])                     # tweets devel
X_test_we_concatenacion = np.array([x[1] for x in test_set_we_concatenacion])                       # tweets test
X_train_we_concatenacion = np.array([x[1] for x in train_set_we_concatenacion])                     # tweets train

#################################### ETIQUETAS ##########################################

y_train_lexicos_we_promedio = [x[2] for x in train_set_lexicos_we_promedio]                 # etiquetas train + léxicos
y_devel_we_promedio = [x[2] for x in devel_set_we_promedio]                                 # etiquetas devel
y_test_we_promedio = [x[2] for x in test_set_we_promedio]                                   # etiquetas test
y_train_we_promedio = [x[2] for x in train_set_we_promedio]                                 # etiquetas train


# Parte 3 - Clasificación de los tweets

Para la clasificación de los tweets es posible trabajar con dos enfoques diferentes:

* Aprendizaje Automático basado en atributos: se pide probar al menos dos modelos diferentes, por ejemplo, Multi Layer Perceptron ([MLP](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)) y Support Vector Machines ([SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)), y usar al menos dos formas de representación de tweets (una basada en BoW y otra basada en word embeddings). Se publicó en eva un léxico de palabras positivas y negativas que puede ser utilizado para generar atributos.

* Aprendizaje Profundo: se recomienda experimentar con alguna red recurrente como LSTM. En este caso deben representar los tweets an base a word embeddings.

Deberán usar el corpus de desarrollo (devel.csv) para comparar resultados de diferentes experimentos, variando los valores de los hiperparámetros, la forma de representación de los tweets, el preprocesamiento, los modelos de AA, etc.

Tanto para la evaluación sobre desarrollo como para la evaluación final sobre test se usará la medida [Macro-F1](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) (promedio de la medida F1 de cada clase).

Imports necesarios:

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler                      #No deberia ser util -> tampoco
from sklearn.model_selection import train_test_split, GridSearchCV   #No deberia ser util -> tampoco
import pandas as pd                                     #Revisar utilidad -> Tampoco
import matplotlib.pyplot as plt                         #Lo mismo -> No lo uso
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris                  #Revisar -> se usaba en versiones viejas de LSTM
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape
from keras.optimizers import RMSprop
from time import time

Declaramos funciones de utilidad para evaluar los clasificadores

In [ ]:
#codificar_etiquetas(y): y es una lista con etiquetas, de la forma ['P','N','NONE','P, ...] donde cada etiqueta es un string que
#puede ser 'P', 'N' o 'NONE', retorna un np.array que codifica las etiquetas a numeros segun el mapeo:
#'P': 0, 'N': 1, 'NONE': 2
def codificar_etiquetas(y):
  etiquetas = {'P': 0, 'N': 1, 'NONE': 2}
  return np.array([etiquetas[i] for i in y])

#imprimir_resultados(f1_scores) -> f1_scores es una tripla de índices, imprime los porcentajes Positivos, Negativos, Neutros
def imprimir_resultados(puntajes):
  print("F1 (P):    " +str(puntajes[0]))
  print("F1 (N):    " +str( puntajes[1]))
  print("F1 (NONE): " + str(puntajes[2]))
  print("Macro-F1: " + str((puntajes[2] + puntajes[1] + puntajes[0])/3))

## MLP

### Bag of Words

#### BOW combinado con TF-IDF:

Primero realizamos pruebas para determinar los mejores valores de ciertos hiperparámetros a ser utilizados para obtener las mejores métricas posibles con este método de aprendizaje automático basado en atributos.

In [ ]:
model_mlp = MLPClassifier(random_state=1234)
param_dict = {'hidden_layer_sizes': [(1,), (3,), (10,)],
              'activation': ['relu', 'logistic'],
              'alpha': [0.001, 0.01]}
import warnings
with warnings.catch_warnings():             # Ignoramos advertencias dentro del bloque with
    warnings.filterwarnings("ignore")
    start = time()
    grid_search = GridSearchCV(model_mlp, param_dict, scoring='f1_macro')
    grid_search.fit(X_train_bowP, y_train_mod)
    print("El tiempo requerido para completar la GridSearch es de %.2f segundos." % (time()-start))
    display(grid_search.best_params_)
    print("Resultado de Cross-Validation (k=5) del mejor estimador: %.3f" % grid_search.best_score_)

El tiempo requerido para completar la GridSearch es de 3452.12 segundos.


{'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': (3,)}

Resultado de Cross-Validation (k=5) del mejor estimador: 0.407


A continuación ejecutamos con distintos valores, donde podemos ver que efectivamente se cumple que la combinación hidden_layer_sizes = 3, activation = "logistic" y alpha = 0.01 es de la que logra mejores resultados, al igual que cuando usamos los mismos datos pero con valor 20 para hidden_layer_sizes. No agregamos pruebas con activation="relu" ya que el rendimiento era bajo.

In [ ]:
def mlp (Ocultas, Alfa, Activacion):
  if Alfa == 0:    # No se usan los hiperparametros "alpha" ni "activation"
    clasificador_mlpAux = MLPClassifier(hidden_layer_sizes=(Ocultas,), max_iter=10000,random_state=30)
  else:
    clasificador_mlpAux = MLPClassifier(hidden_layer_sizes=(Ocultas,), max_iter=10000,random_state=30, alpha=Alfa,activation=Activacion)
  clasificador_mlpAux.fit(X_train_bowP, y_train_mod)
  y_pred_devel_BOW_MLP = clasificador_mlpAux.predict(X_devel_bowP)           #BOWs de Corpus Test pre-procesado
  average_def = None
  f1_devel_BOW_MLP = f1_score(y_devel, y_pred_devel_BOW_MLP,average="macro")
  return f1_devel_BOW_MLP

In [ ]:
print("Macro-F1 para hidden_layer_sizes=3: ", mlp(3,0,""))
print("Macro-F1 para hidden_layer_sizes=10: ", mlp(10,0,""))
print("Macro-F1 para hidden_layer_sizes=20: ", mlp(20,0,""))
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.01 y activation=logistic: ", mlp(3,0.01,"logistic"))
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.001 y activation=logistic: ", mlp(3,0.001,"logistic"))
print("Macro-F1 para hidden_layer_sizes=10, alpha=0.01 y activation=logistic: ", mlp(10,0.01,"logistic"))
print("Macro-F1 para hidden_layer_sizes=10, alpha=0.001 y activation=logistic: ", mlp(10,0.001,"logistic"))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.01 y activation=logistic: ", mlp(20,0.01,"logistic"))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.001 y activation=logistic: ", mlp(20,0.001,"logistic"))

Macro-F1 para hidden_layer_sizes=3:  0.5128484176918356
Macro-F1 para hidden_layer_sizes=10:  0.5359836285230957
Macro-F1 para hidden_layer_sizes=20:  0.5134017494869968
Macro-F1 para hidden_layer_sizes=3, alpha=0.01 y activation=logistic:  0.5539079436598462
Macro-F1 para hidden_layer_sizes=3, alpha=0.001 y activation=logistic:  0.5159206945279009
Macro-F1 para hidden_layer_sizes=10, alpha=0.01 y activation=logistic:  0.5531464568317209
Macro-F1 para hidden_layer_sizes=10, alpha=0.001 y activation=logistic:  0.5220898448075021
Macro-F1 para hidden_layer_sizes=20, alpha=0.01 y activation=logistic:  0.5557007990001974
Macro-F1 para hidden_layer_sizes=20, alpha=0.001 y activation=logistic:  0.5168146446810935


Como se puede observar, es levemente mejor, por lo que decidimos a partir de este momento no solo consideraremos dichos valores obtenidos en la prueba basada en Cross-Validation para MLP, sino que recurriremos a ensayo y error en busca de los mejores resultados posibles.

#### BOW para k-features mas relevantes:

A cada BOW que representa un tweet, se le agrega a dicha representación dos valores mas al final de este, de modo que en la anteúltima coordenada tendremos el numero de palabras en el tweet que se encuentran presentes en el léxico de palabras positivas. Del mismo modo la ultima coordenada del vector tendra la cantidad de palabras del tweet que se encuentran en el léxico de palabras negativas.

Definimos una funcion a continuación para estudiar como se comporta MLP ante tal representación de los tweets.

In [ ]:
def mlp_bow_k (Ocultas, Alfa, Activacion,ValorK):
  if Alfa == 0:    # No se usan los hiperparametros "alpha" ni "activation"
    clasificador_mlp_bowk1 = MLPClassifier(hidden_layer_sizes=(Ocultas,), max_iter=10000,random_state=30)
  else:
    clasificador_mlp_bowk1 = MLPClassifier(hidden_layer_sizes=(Ocultas,), max_iter=10000,random_state=30, alpha=Alfa,activation=Activacion)
  vocabulario = seleccionarK(X_trainP,y_train_mod,ValorK)

  X_train_bow_k_mlp = representar_con_k(X_trainP,vocabulario)
  #extras = lemas_Conjunto(X_trainP)
  X_train_bow_k_mlp_extendido = agregar_a_bow(X_train_bow_k_mlp,extras)

  clasificador_mlp_bowk1.fit(X_train_bow_k_mlp_extendido, y_train_mod)

  X_devel_bow_k_mlp = representar_con_k(X_develP,vocabulario)
  #extras2 = lemas_Conjunto(X_develP)
  X_devel_bow_k_mlp_extendido = agregar_a_bow(X_devel_bow_k_mlp,extras2)

  y_pred_test_BOW_K_MLP = clasificador_mlp_bowk1.predict(X_devel_bow_k_mlp_extendido)           #BOWs de Corpus Test pre-procesado
  average_def = None

  f1_devel_BOW_K_MLP = f1_score(y_devel, y_pred_test_BOW_K_MLP,average="macro")
  return f1_devel_BOW_K_MLP

In [ ]:
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistib y k=250: ", mlp_bow_k(3,0.01,"logistic",250))
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=500: ", mlp_bow_k(3,0.01,"logistic",500))
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=1000: ", mlp_bow_k(3,0.01,"logistic",1000))
print("Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=250: ", mlp_bow_k(8,0.01,"logistic",250))
print("Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=500: ", mlp_bow_k(8,0.01,"logistic",500))
print("Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=1000: ", mlp_bow_k(8,0.01,"logistic",1000))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=250: ", mlp_bow_k(20,0.01,"logistic",250))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=500: ", mlp_bow_k(20,0.01,"logistic",500))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=1000: ", mlp_bow_k(20,0.01,"logistic",1000))

Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistib y k=250:  0.5472966598294682
Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=500:  0.5615829079500244
Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=1000:  0.5542923619111257
Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=250:  0.5399119892961677
Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=500:  0.5580513107947079
Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=1000:  0.5530611064704959
Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=250:  0.5449202561118403
Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=500:  0.556950140704333
Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=1000:  0.5073824397096428


Nuevamente los mejores valores de Macro-F1 se atribuyen a hidden_layer_sizes = 3, alpha = 0.01 y activation = "logistic". A su vez se destaca que el mejor rendimiento del modelo es cuando agregamos las dos coordendas a los BOWs con las 500 features mas relevantes del Data Set de entrenamiento, con Macro-F1 = 0.562. A su vez, con este enfoque se reduce significativamente el tamaño de los vectores: de dimensión mayor a 21.000 con el enfoque TF-IDF a 500 para el caso anterior con enfoque Select k-best, obteniendo mejores resultados.

Veamos ahora que sucede si no agregamos las dos coordenadas extras:

In [ ]:
def mlp_bow_k_sin_extras (Ocultas, Alfa, Activacion,ValorK):
  if Alfa == 0:    # No se usan los hiperparametros "alpha" ni "activation"
    clasificador_mlp_bowk2 = MLPClassifier(hidden_layer_sizes=(Ocultas,), max_iter=10000,random_state=30)
  else:
    clasificador_mlp_bowk2 = MLPClassifier(hidden_layer_sizes=(Ocultas,), max_iter=10000,random_state=30, alpha=Alfa,activation=Activacion)
  vocabulario = seleccionarK(X_trainP,y_train_mod,ValorK)
  X_train_bow_k_mlp2 = representar_con_k(X_trainP,vocabulario)

  clasificador_mlp_bowk2.fit(X_train_bow_k_mlp2, y_train_mod)

  X_devel_bow_k_mlp2 = representar_con_k(X_develP,vocabulario)

  y_pred_test_BOW_K_MLP2 = clasificador_mlp_bowk2.predict(X_devel_bow_k_mlp2)           #BOWs de Corpus Test pre-procesado
  average_def = None

  f1_devel_BOW_K_MLP2 = f1_score(y_devel, y_pred_test_BOW_K_MLP2,average="macro")
  return f1_devel_BOW_K_MLP2

In [ ]:
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=250: ", mlp_bow_k_sin_extras(3,0.01,"logistic",250))
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=500: ", mlp_bow_k_sin_extras(3,0.01,"logistic",500))
print("Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=1000: ", mlp_bow_k_sin_extras(3,0.01,"logistic",1000))
print("Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=250: ", mlp_bow_k_sin_extras(8,0.01,"logistic",250))
print("Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=500: ", mlp_bow_k_sin_extras(8,0.01,"logistic",500))
print("Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=1000: ", mlp_bow_k_sin_extras(8,0.01,"logistic",1000))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=250: ", mlp_bow_k_sin_extras(20,0.01,"logistic",250))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=500: ", mlp_bow_k_sin_extras(20,0.01,"logistic",500))
print("Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=1000: ", mlp_bow_k_sin_extras(20,0.01,"logistic",1000))

Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistib y k=250:  0.5472966598294682
Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=500:  0.5615829079500244
Macro-F1 para hidden_layer_sizes=3, alpha=0.01, activation=logistic y k=1000:  0.5542923619111257
Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=250:  0.5399119892961677
Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=500:  0.5580513107947079
Macro-F1 para hidden_layer_sizes=8, alpha=0.01, activation=logistic y k=1000:  0.5530611064704959
Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=250:  0.5449202561118403
Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=500:  0.556950140704333
Macro-F1 para hidden_layer_sizes=20, alpha=0.01, activation=logistic y k=1000:  0.5073824397096428


Viendo la salida anterior, resulta que si agregamos las dos coordendas extras obtenemos los mismos resultados que al no hacerlo.

Vemos que usando 8 capas ocultas, alpha = 0.01 y función de activación logistic, obtenemos un mejor rendimiento que utilizando BOWs combinados con Tf-Idf, para vectores de menor tamaño. Por lo tanto, sin agregar las coordendas es conveniente utilizar el enfoque de selección de las 500 características mas relevantes del Corpus Train.

### Word Embeddings:

Codificamos las etiquetas de los corpus.

In [ ]:
y_train_lexicos_numerico_we = codificar_etiquetas(y_train_lexicos_we_promedio)
y_devel_numerico_we         = codificar_etiquetas(y_devel_we_promedio)
y_test_numerico_we          = codificar_etiquetas(y_test_we_promedio)
y_train_numerico_we         = codificar_etiquetas(y_train_we_promedio)

#### Word embeddings tomando el promedio:
Para esta parte vamos a utilizar word embeddings de tamaño de vector 302, en donde los primeros 300 valores de cada vector de la entrada son el promedio de los vectores obtenidos de la colección importada y los últimos 2 valores son la cantidad de palabras positivas y negativas identificadas en cada tweet.

Fase de testeo para obtener los mejores parámetros para la función:

In [ ]:
mlp_we_promedio = MLPClassifier(random_state=20)
param_dict = {'hidden_layer_sizes': [(1,), (3,), (10,),(100,),(200,)],
              'activation': ['relu', 'logistic'],
              'alpha': [0.0001, 0.001, 0.01, 0.1],
              'max_iter':[1000]}
start = time()
grid_search = GridSearchCV(mlp_we_promedio, param_dict, scoring='f1_macro')
grid_search.fit(X_train_lexicos_we_promedio, y_train_lexicos_numerico_we)
print("El tiempo requerido para completar la GridSearch fué de %.2f segundos." % (time()-start))
display(grid_search.best_params_)
print("Resultado de Cross-Validation (k=5) del mejor estimador: %.3f" % grid_search.best_score_)

Creamos el clasificador con los mejores parámetros obtenidos de la parte anterior:

In [ ]:
mlp_we_promedio = MLPClassifier(hidden_layer_sizes=(200,), max_iter=1000,alpha = 0.01, random_state=20,activation='relu')
mlp_we_promedio.fit(X_train_lexicos_we_promedio, y_train_lexicos_numerico_we)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(200,), max_iter=1000,
              random_state=20)

In [ ]:
##################################################################
#####################  Cálculo de medida F1  #####################
##################################################################


#Hacemos predicciones en el conjunto de prueba:
y_pred_devel_mlp_we = mlp_we_promedio.predict(X_devel_we_promedio)

#Calculamos los Puntajes f1
MLP_f1_devel = f1_score(y_devel_numerico_we, y_pred_devel_we,average=None)

#Mostramos los resultados en consola
print("Resultados en el corpus devel: ")
imprimir_resultados(MLP_f1_devel)
print("\n")

Resultados en el corpus devel: 
F1 (P):    0.6469194312796208
F1 (N):    0.5874125874125874
F1 (NONE): 0.5078014184397163
Macro-F1: 0.5807111457106414




#### Word Embeddings tomando la concatenación:
Ahora, vamos a utilizar word embeddings de tamaño de vector $300 \times M+2$,
siendo M la cantidad máxima de palabras en cada tweet, esta se encuentra definida como 10 mas arriba, en la función de la parte 2. Tomamos este valor ya que es el promedio de las palabras de los tweets del corpus train_set.

In [ ]:
mlp_we_concatenacion = MLPClassifier(hidden_layer_sizes=(200,), max_iter=1000,alpha = 0.01, random_state=20,activation='relu')
mlp_we_concatenacion.fit(X_train_lexicos_we_concatenacion, y_train_lexicos_numerico_we)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(200,), max_iter=1000,
              random_state=20)

In [ ]:
##################################################################
#####################  Cálculo de medida F1  #####################
##################################################################

#Hacemos predicciones en el conjunto de prueba:
y_pred_devel_we_mlp_concat = mlp_we_concatenacion.predict(X_devel_we_concatenacion)
y_pred_test_we_mlp_concat = mlp_we_concatenacion.predict(X_test_we_concatenacion)

average_def = None


#Calculamos los Puntajes f1
MLP_f1_devel = f1_score(y_devel_numerico_we, y_pred_devel_we_mlp_concat,average=average_def)
mlp4_we_concatenacion = f1_score(y_test_numerico_we, y_pred_test_we_mlp_concat,average=average_def)

parte4_we.append(['MLP Concatenación: ',mlp4_we_concatenacion])

#Mostramos los resultados en consola
print("Resultados en el corpus devel: ")
imprimir_resultados(MLP_f1_devel)
print("\n")

Resultados en el corpus devel: 
F1 (P):    0.6004962779156328
F1 (N):    0.5770862800565771
F1 (NONE): 0.5059920106524634
Macro-F1: 0.561191522874891




Observamos:
- Resultados
  - La macro F1 de MLP utilizando word embeddings tomando la concatenación es aproximadamente 0.56
  - La macro F1 de MLP utilizando word embeddings tomando el promedio es aproximadamente 0.58
- Concluimos en esta parte que la representación de promedio es mejor tanto en velocidad de procesamiento como en eficacia del análisis de sentimientos, ya que obtiene mejores resultados en tiempos de ejecución menores, aunque el modelo de word embeddings tomando la concatenación tiene capacidad de mejora, ya que sabemos que hay tweets de hasta 22 palabras en nuestro corpus, mientras que solo estamos tomando las 10 primeras que se encuentran identificadas en nuestra colección de vectores.

## SVM

### Bag of Words

#### BOW combiando con TF-IDF:

Primero realizamos pruebas para determinar los mejores valores de ciertos hiperparámetros a ser utilizados para otener las mejores métricas posibles con este método de aprendizaje automático basado en atributos.

In [ ]:
model_svm2 = svm.SVC(random_state=1234)
param_dict = {'C': [ 5, 10, 20],
             'kernel': ['linear', 'sigmoid']}

start = time()
grid_search = GridSearchCV(model_svm2, param_dict, scoring='f1_macro')
grid_search.fit(X_train_bowP, y_train_mod)
print("El tiempo requerido para completar la GridSearch es de %.2f segundos." % (time()-start))
display(grid_search.best_params_)
print("Resultado de Cross-Validation (k =5) del mejor estimador: %.3f" % grid_search.best_score_)

El tiempo requerido para completar la GridSearch es de 1033.73 segundos.


{'C': 5, 'kernel': 'linear'}

Resultado de Cross-Validation (k =5) del mejor estimador: 0.386


Como el mejor estimador proporciona una Macro-F1 de 0.386, declararemos un clasificador en función de dichos valores de hiperparámetros que permitieron alcanzarla luego de aplicar Cross-Validation en el corpus de Entrenamiento, pero tambien probaremos con otros valores debido a que dicho valor de Macro-F1 es bajo.

In [ ]:
# Experimentos con Aprendizaje Atuomático y BoW
#SVM
model_svm = svm.SVC(C=5, kernel='linear')
model_svm.fit(X_train_bowP, y_train_mod)

SVC(C=5, kernel='linear')

In [ ]:
model_svm.predict(X_devel_bowP[:10])    #Mostramos primeras 10 predicciones del Clasificador SVM.

array(['P', 'N', 'P', 'NONE', 'P', 'N', 'N', 'N', 'P', 'N'], dtype='<U4')

In [ ]:
y_pred_devel_BOW_SVM= model_svm.predict(X_devel_bowP)
f1_devel_BOW_SVM = f1_score(y_devel, y_pred_devel_BOW_SVM,average="macro")
print("Macro-F1: ",f1_devel_BOW_SVM)

Macro-F1:  0.5730006777836226


A continuación probaremos el comportamiento del modelo de clasificación SVM para tres valores distintos de C: 3, 8 y 20. Tambien tres funciones de transformación de entrada diferentes: linear, rbf y sigmoid.

In [ ]:
def svmBow(ValorC,ValorKernel):
  model_svmBowAux = svm.SVC(C=ValorC, kernel=ValorKernel)
  model_svmBowAux.fit(X_train_bowP, y_train_mod)
  y_pred_devel_BOW_SVM= model_svmBowAux.predict(X_devel_bowP)
  f1_devel_BOW_SVM = f1_score(y_devel, y_pred_devel_BOW_SVM,average="macro")
  return f1_devel_BOW_SVM

In [ ]:
print("Macro-F1  para C=3 y kernel=linear: ",svmBow(3,"linear"))
print("Macro-F1  para C=3 y kernel=rbf: ",svmBow(3,"rbf"))
print("Macro-F1  para C=3 y kernel=sigmoid: ",svmBow(3,"sigmoid"))
print("Macro-F1  para C=8 y kernel=linear: ",svmBow(8,"linear"))
print("Macro-F1  para C=8 y kernel=rbf: ",svmBow(8,"rbf"))
print("Macro-F1  para C=8 y kernel=sigmoid: ",svmBow(8,"sigmoid"))
print("Macro-F1  para C=20 y kernel=linear: ",svmBow(20,"linear"))
print("Macro-F1  para C=20 y kernel=rbf: ",svmBow(20,"rbf"))
print("Macro-F1  para C=20 y kernel=sigmoid: ",svmBow(20,"sigmoid"))

Macro-F1  para C=3 y kernel=linear:  0.5817227225718874
Macro-F1  para C=3 y kernel=rbf:  0.5900978362201837
Macro-F1  para C=3 y kernel=sigmoid:  0.5823754773523954
Macro-F1  para C=8 y kernel=linear:  0.5614140697110582
Macro-F1  para C=8 y kernel=rbf:  0.5910828991322029
Macro-F1  para C=8 y kernel=sigmoid:  0.5590214352425277
Macro-F1  para C=20 y kernel=linear:  0.5433629548223026
Macro-F1  para C=20 y kernel=rbf:  0.5910828991322029
Macro-F1  para C=20 y kernel=sigmoid:  0.5461235453515433


De la anterior salida, vemos que el valor de kernel = "rbf" arroja los mejores resultados, para distintos valores de C. Por lo tanto veremos si es aun posible mejorar tales performance, al incluir el hiperparámetro gamma que es un hiperparámetro de ajuste que controla la influencia de cada muestra en el modelo. Dicho valor suele combinarse con "rbf". Dada la salida anterior solo consideraremos C = 3 ya que arrojo buenos resultados en general.

In [ ]:
def svmBowGamma(ValorC,ValorKernel,Gamma):
  model_svmBowAux = svm.SVC(C=ValorC, kernel=ValorKernel,gamma=Gamma)
  model_svmBowAux.fit(X_train_bowP, y_train_mod)
  y_pred_devel_BOW_SVM= model_svmBowAux.predict(X_devel_bowP)
  f1_devel_BOW_SVM = f1_score(y_devel, y_pred_devel_BOW_SVM,average="macro")
  return f1_devel_BOW_SVM

In [ ]:
print("Macro-F1  para C=3, kernel=rbf y gamma=0.1: ",svmBowGamma(3,"rbf",0.1))
print("Macro-F1  para C=3, kernel=rbf y gamma=1: ",svmBowGamma(3,"rbf",1))
print("Macro-F1  para C=3, kernel=rbf y gamma=2: ",svmBowGamma(3,"rbf",2))
print("Macro-F1  para C=3, kernel=rbf y gamma=5: ",svmBowGamma(3,"rbf",5))

Macro-F1  para C=3, kernel=rbf y gamma=0.1:  0.5965084127696065
Macro-F1  para C=3, kernel=rbf y gamma=1:  0.5880618662434957
Macro-F1  para C=3, kernel=rbf y gamma=2:  0.5976970560303894
Macro-F1  para C=3, kernel=rbf y gamma=5:  0.2978999398781148


Se obtienen leves mejoras respecto a la ejecución anterior al considerar distintos valores de gamma.

Como el uso de BOWs combinados con Tf-Idf para representar los tweets en un modelo SVM con C = 3, kernel = "rbf" y gamma = 2 es uno de los enfoques que ha logrado mejores resultados, ejecutamos nuevamente para determinar la F1 de cada clase:

In [ ]:
model_svmBowCandidato = svm.SVC(C=3, kernel= "rbf",gamma= 2)
model_svmBowCandidato.fit(X_train_bowP, y_train_mod)
y_pred_devel_BOW_SVM_C= model_svmBowCandidato.predict(X_devel_bowP)
f1_devel_BOW_SVM_Cand = f1_score(y_devel, y_pred_devel_BOW_SVM_C,average=None)
imprimir_resultados(f1_devel_BOW_SVM_Cand)

F1 (P):    0.6296296296296297
F1 (N):    0.5
F1 (NONE): 0.6634615384615384
Macro-F1: 0.5976970560303894


Estos resultados seran mencionados en la pregunta 7.

#### BOW para k-features mas relevantes:

Al igual que como se hizo con MLP, agregamos dos coordendas extras. Como anteriormente se obtuvieron los mejores resultados para kernel = "rbf", consideramos a continuación dicha función de transformación. A su vez probamos el comportamiento para distintos valores de k.

In [ ]:
def svmBowK_ext(ValorC,ValorKernel,ValorK):
  vocabularioSVM = seleccionarK(X_trainP,y_train_mod,ValorK)
  X_train_bow_k_svm = representar_con_k(X_trainP,vocabularioSVM)
  X_train_bow_k_svm_extendido = agregar_a_bow(X_train_bow_k_svm,extras)     # Agregamos dos coordenadas extras a BOWs de train

  model_svmBowAux = svm.SVC(C=ValorC, kernel=ValorKernel)
  model_svmBowAux.fit(X_train_bow_k_svm_extendido, y_train_mod)

  X_devel_bow_k_svm = representar_con_k(X_develP,vocabularioSVM)
  X_devel_bow_k_svm_extendido = agregar_a_bow(X_devel_bow_k_svm,extras2)    # Agregamos dos coordenadas extras a BOWs de devel

  y_pred_devel_BOW_K_SVM= model_svmBowAux.predict(X_devel_bow_k_svm_extendido)
  f1_devel_BOW_K_SVM = f1_score(y_devel, y_pred_devel_BOW_K_SVM,average="macro")
  return f1_devel_BOW_K_SVM

In [ ]:
print("Macro-F1  para C=3, kernel=rbf y k=250: ",svmBowK_ext(3,"rbf",250))
print("Macro-F1  para C=3, kernel=rbf y k=500: ",svmBowK_ext(3,"rbf",500))
print("Macro-F1  para C=3, kernel=rbf y k=1000: ",svmBowK_ext(3,"rbf",1000))
print("Macro-F1  para C=8, kernel=rbf y k=250: ",svmBowK_ext(8,"rbf",250))
print("Macro-F1  para C=8, kernel=rbf y k=500: ",svmBowK_ext(8,"rbf",500))
print("Macro-F1  para C=8, kernel=rbf y k=1000: ",svmBowK_ext(8,"rbf",1000))
print("Macro-F1  para C=20, kernel=rbf y k=250: ",svmBowK_ext(20,"rbf",250))
print("Macro-F1  para C=20, kernel=rbf y k=500: ",svmBowK_ext(20,"rbf",500))
print("Macro-F1  para C=20, kernel=rbf y k=1000: ",svmBowK_ext(20,"rbf",1000))

Macro-F1  para C=3, kernel=rbf y k=250:  0.5145076263796833
Macro-F1  para C=3, kernel=rbf y k=500:  0.5304865230779972
Macro-F1  para C=3, kernel=rbf y k=1000:  0.5285714630418378
Macro-F1  para C=8, kernel=rbf y k=250:  0.5144459242299119
Macro-F1  para C=8, kernel=rbf y k=500:  0.5232950859348796
Macro-F1  para C=8, kernel=rbf y k=1000:  0.5214419252393935
Macro-F1  para C=20, kernel=rbf y k=250:  0.5144459242299119
Macro-F1  para C=20, kernel=rbf y k=500:  0.5223809368786031
Macro-F1  para C=20, kernel=rbf y k=1000:  0.5175840110865974


Analizando la salida vemos que los mejores resultados se corresponden cuando k = 500, con C = 3, donde recordemos los BOWs de de los tweets tenian dos coordenadas extras que contemplaban la cantidad de palabras positivas en la anteúltima coordenada, y negativas en la última.

Por otro lado, si utilizamos kernel = "linear", a diferencia del enfoque de BOWs combinados con TF-IDF, tenemos que "linear" arroja mejores resultados que "rbf", considerando k = 500 que fue el de mejor rendimiento en las celdas anterores:

In [ ]:
print("Macro-F1  para C=3, kernel=linear y k=500: ",svmBowK_ext(3,"linear",500))
print("Macro-F1  para C=8, kernel=linear y k=500: ",svmBowK_ext(8,"linear",500))
print("Macro-F1  para C=20, kernel=linear y k=500: ",svmBowK_ext(20,"linear",500))

Macro-F1  para C=3, kernel=linear y k=500:  0.5492594510453775
Macro-F1  para C=8, kernel=linear y k=500:  0.5429893572237808
Macro-F1  para C=20, kernel=linear y k=500:  0.5387224285971939


Por último veamos que sucede si no agregamos las dos coordendas extras, con k = 500, C = 5 y kernel = "linear":

In [ ]:
model_svm_sin_ext = svm.SVC(C=3, kernel='linear')
vocabularioSVM_sin_ext = seleccionarK(X_trainP,y_train_mod,2000)

X_train_bow_k_svm_sin_ext = representar_con_k(X_trainP,vocabularioSVM_sin_ext)

model_svm_sin_ext.fit(X_train_bow_k_svm_sin_ext, y_train_mod)

X_devel_bow_k_svm3 = representar_con_k(X_develP,vocabularioSVM_sin_ext)

y_pred_devel_BOW_K_SVM3 = model_svm_sin_ext.predict(X_devel_bow_k_svm3)
f1_devel_BOW_K_SVM3 = f1_score(y_devel, y_pred_devel_BOW_K_SVM3,average="macro")
print("Macro-F1  para C=3, kernel=linear y k=500: ",f1_devel_BOW_K_SVM3)

Macro-F1  para C=3, kernel=linear y k=500:  0.5330609615939691


Vemos que el rendimiento es practicamente igual al obtenido con C = 3, kernel = "rbf", k = 500 y agregando las dos coordenadas. Entonces, en un principio, el uso de las mismas no implica primariamente mejores resultados.

En resumen, SVM trabaja mejor con BOWs combinados con Tf-Idf, aunque dicha representación requiere mayor espacio de almacenamiento debido a la gran diferencia entre las dimensiones de los vectores. Por lo tanto en la práctica habría que analizar si dicho costo se puede asumir en busca de mejores resultados. Sin embargo, se suele optar por redes neuronales basadas en la arquitectura transformers como veremeos en la parte 4.

### Word Embeddings

#### Word embeddings promedio

Para esta parte se probó:
- En una primera instancia con varios valores distintos para el parámetro 'C', obteniendo como valores altos como los más apropiados.
- Luego, se verificó para varios tipos de núcleo ajustando el parámetro 'kernel' como 'linear' o 'sigmoide', así como gaussiano ('rbf') ajustando el parámetro gamma y polinomial ('poly') ajustando el parámetro 'degree'.
- Finalmente probamos variando tanto el corpus de entrada (train puro o train con los léxicos concatenados) y la variable class_weight ('balanced' o por defecto).\
De los resultados anteriores, presentamos el mejor clasificador obtenido:\
Para corpus train puro, es decir, sin léxicos agregados como tweets, con los parámetros C=70, kernel = 'linear' y class_weight balanceado.

In [ ]:
######################################################
################ SVM con Word embeddings:#############
######################################################

svm_we_prom = svm.SVC(C=63, kernel='linear',class_weight='balanced')
svm_we_prom.fit(X_train_we_promedio, y_train_numerico_we)

SVC(C=30, class_weight='balanced', kernel='linear')

In [ ]:
# cross-validation de 5 etapas:
#Solo utilizamos el corpus train, sin agregar el léxico.

model_svm_acc = cross_val_score(estimator=svm_we_prom, X=X_train_we_promedio, y=y_train_numerico_we, cv=5, n_jobs=-1)
print('Accuracy en c-v de 5 etapas:' + str(model_svm_acc))

Accuracy en c-v de 5 etapas:[0.60853879 0.60012026 0.61395069 0.5694528  0.59386282]


In [ ]:
##################################################################
#####################  Cálculo de medida F1  #####################
##################################################################

y_pred_devel_svm_we = svm_we_prom.predict(X_devel_we_promedio)
svm_f1_devel = f1_score(y_devel_numerico_we, y_pred_devel_svm_we, average=None)

print("Resultados en el corpus devel: ")
imprimir_resultados(svm_f1_devel)
print("\n")

Resultados en el corpus devel: 
F1 (P):    0.6536964980544746
F1 (N):    0.625514403292181
F1 (NONE): 0.5130890052356021
Macro-F1: 0.5974333021940859




#### Word embeddings concatenación:

In [ ]:
svm_we_concatenacion = SVC(C=22, kernel='linear')
svm_we_concatenacion.fit(X_train_we_concatenacion, y_train_numerico_we)

SVC(C=22, kernel='linear')

In [ ]:
##################################################################
#####################  Cálculo de medida F1  #####################
##################################################################

y_pred_devel_svm_we = svm_we_concatenacion.predict(X_devel_we_concatenacion)
y_pred_test_svm_we = svm_we_concatenacion.predict(X_test_we_concatenacion)

average_def = None

svm_f1_devel = f1_score(y_devel_numerico_we, y_pred_devel_svm_we, average=average_def)

print("Resultados en el corpus devel: ")
imprimir_resultados(svm_f1_devel)
print("\n")

Resultados en el corpus devel: 
F1 (P):    0.5775656324582339
F1 (N):    0.5142002989536623
F1 (NONE): 0.47556142668428003
Macro-F1: 0.5224424526987254




## Naive Bayes

A modo de experimentación, veremos como se comporta el modelo de aprendizaje automático basado en atributos Naive Bayes para BOWs combinados con Tf-Idf. No lo haremos con WE ya que estos son representaciones vectoriales densas de palabras que capturan el significado semántico y la relación entre las palabras. Mientras que normalmente  Naive Bayes se utiliza con características basadas en recuentos de palabras, como la frecuencia de términos o la presencia/ausencia de palabras específicas en un documento. Es decir, es mas conveniente para representaciones basadas en BOW. Por otro lado los resultados no son alentadores como para tenerlo mas en consideración.

In [ ]:
from sklearn.naive_bayes import GaussianNB
model_gnb = GaussianNB()
model_gnb.fit(X_train_bowP.toarray(), y_train_mod)
y_predict_NB = model_gnb.predict(X_devel_bowP.toarray())
res_NB = f1_score(y_devel, y_predict_NB, average=None)
imprimir_resultados(res_NB)

F1 (P):    0.4714285714285714
F1 (N):    0.2576271186440678
F1 (NONE): 0.5215605749486653
Macro-F1: 0.4168720883404348


## Regresión Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

Al igual que con Naive Bayes, armamos un simple clasificador para este modelo para experimentar como responde a esta tarea de PLN. Consideramos utilizar los tweets en representacion BOW, y también en WE Promedio.

### Bag Of Words

In [ ]:
model_lg = LogisticRegression()
model_lg.fit(X_train_bowP.toarray(), y_train_mod)
y_predict_LG = model_lg.predict(X_devel_bowP.toarray())
res_LG = f1_score(y_devel, y_predict_LG, average=None)
imprimir_resultados(res_LG)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 (P):    0.6048387096774194
F1 (N):    0.466765140324963
F1 (NONE): 0.6548042704626335
Macro-F1: 0.5754693734883386


### Word embeddings Promedio

In [ ]:
model_lg_we = LogisticRegression(C=0.1,max_iter=1000)
model_lg_we.fit(X_train_we_promedio, y_train_numerico_we)
y_predict_LG = model_lg_we.predict(X_devel_we_promedio)
res_LG = f1_score(y_devel_numerico_we, y_predict_LG, average=None)
imprimir_resultados(res_LG)

F1 (P):    0.6626065773447015
F1 (N):    0.6100278551532033
F1 (NONE): 0.5324137931034483
Macro-F1: 0.6016827418671178


Como podemos ver, los resultados de esta parte son bastante buenos. En BoW la clase que obtiene peor F1 es "N", y la mejor "NONE". Por otro lado, en WE obtenemos un peor F1 para la clase NONE, mientras que los de clase P y N tienen resultados ampliamente superiores.

## LSTM

Imports necesarios para esta parte:

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import GlobalMaxPool2D, BatchNormalization

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 9.4 MB/s eta 0:00:00


In [ ]:
import tensorflow_addons as tfa
f1_macro = tfa.metrics.F1Score(num_classes=3, average='macro')

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


### LSTM sin atributos en los Word Embeddings

Primero definimos una función que determina el largo mas grande de un tweet en un corpus (procesado):

In [ ]:
def largo_maximo(corpus):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(corpus)
  sequence_lengths = [len(sequence.split()) for sequence in corpus] # Obtenemos las longitudes de las secuencias
  max_length = max(sequence_lengths)
  return max_length

max_length_train = largo_maximo(X_trainP)
max_length_devel = largo_maximo(X_develP)
print(max_length_train)
print(max_length_devel)

21
18


A continuacion, obtenemos una matriz con los WordEmbeddings de las palabras que aparecen en el Corpus de entrenamiento. De esta forma aprovechamos WE pre-entrenados en una capa Embedding de la red LSTM que definiremos proximamente. Tambien llevamos a los conjuntos de datos de entrenamiento y desarrollo al formato adecuado.

In [ ]:
embedding_model = Cool.copy()                            # Cool obtenido en descraga de WE
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_trainP)
vocab_size = len(tokenizer.word_index) + 1

X_train_sequences = tokenizer.texts_to_sequences(X_trainP)   # Convertimos los conjuntos en secuencias de índices
X_devel_sequences = tokenizer.texts_to_sequences(X_develP)

max_length = max_length_train                                # Ajustamos las secuencias a una longitud máxima, la del Data set de entrenamiento
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length)
X_devel_padded = pad_sequences(X_devel_sequences, maxlen=max_length)

embedding_dim = 300                                          # Creamos una matriz de embedding inicializada con ceros
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():                 # Rellenamos la matriz de embedding con los vectores pre-entrenados
    if word in embedding_model:
        embedding_matrix[i] = embedding_model[word]

Llevamos los conjuntos de etiquetas de polaridad a un formato One-Hot ya que trabajamos con 3 tipos diferentes de las mismas.

In [ ]:
sentimientos = ['N', 'P', 'NONE']

def etiquetas_one_hot(etiquetas_originaless):
  etiquetas_esperadas = np.zeros((len(etiquetas_originaless), len(sentimientos)))
  for i, etiqueta in enumerate(etiquetas_originaless):
      index = sentimientos.index(etiqueta)
      etiquetas_esperadas[i, index] = 1
  return etiquetas_esperadas

In [ ]:
y_trainHot = etiquetas_one_hot(y_train_mod)
y_develHot = etiquetas_one_hot(y_devel)

Definimos la primer red LSTM. La misma utiliza los WE pre-entrenados que fueron cargados en Parte 2 y puestos en una matriz anteriormente. En el resumen de la Red, veremos que esta dispone para entrenar mas de 85.000 parámetros, contando con otros 6.600.000 de parámetros aproximadamente no entrenables, pues quedaron determinados por los WE pre-entrenados.  Utilizamos una Capa Bidireccional LSTM para capturar tanto el contexto pasado como el futuro de las secuencias.


Se compila el modelo utilizando el optimizador Adam, la función de pérdida categorical_crossentropy y se utiliza la métrica F1 Score para evaluar el rendimiento.

In [ ]:
def lstm1 (Max_length, X_trainAux, y_trainAux, X_develAux, y_develAux,Mat):
  model = Sequential()
  model.add(Embedding(vocab_size, 300, weights=[Mat],
                      input_length=Max_length, trainable=False))
  model.add(Bidirectional(LSTM(32)))
  model.add(Dense(3, activation='softmax'))      # 3 por las 3 etiquetas posibles, con activation "softmax" que es la adecuada para tal caso
  # Compilamos el modelo, usamos loss = 'categorical_crossentropy' ya que tenemos tres categorias diferentes
  model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=[tfa.metrics.F1Score(average='macro',num_classes=3)], run_eagerly=True)
  model.summary()
  model.fit(X_trainAux, y_trainAux, epochs=5, batch_size=32)          # Entrenamos el modelo
  y_obtenido = model.predict(X_develAux)
  f1_macro = tfa.metrics.F1Score(num_classes=3, average='macro')
  f1_macro.update_state(y_develAux, y_obtenido)
  macro_f1Aux2 = f1_macro.result().numpy()
  return macro_f1Aux2

In [ ]:
macro_lstm1 = lstm1 (21,X_train_padded,y_trainHot,X_devel_padded,y_develHot,embedding_matrix)
print(macro_lstm1)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 21, 300)           6531600   
                                                                 
 bidirectional_14 (Bidirecti  (None, 64)               85248     
 onal)                                                           
                                                                 
 dense_36 (Dense)            (None, 3)                 195       
                                                                 
Total params: 6,617,043
Trainable params: 85,443
Non-trainable params: 6,531,600
_________________________________________________________________
Epoch 1/5
575/575 [==============================] - 124s 214ms/step - loss: 0.9245 - f1_score: 0.5566
Epoch 2/5
575/575 [==============================] - 123s 215ms/step - loss: 0.6879 - f1_score: 0.7209
Epoch 3/5
575/575 [============

Como podemos observar, luego de 5 épocas se alcanza una Macro-F1 de 0.591 aproximadamente. Por otro lado, vemos también que los tiempos requeridos para el entrenamiento no son malos, pues se requiere aproximadamente 2 minutos por época.

Pasamos a definir una nueva red LSTM. Volvemos a entrenar durante 5 epocas. Respecto al modelo anterior, pasamos a utilizar capas más variadas en busca de mejorar la performance. Entre dichas capas, tenemos dos capas densas (Dense) con activación ReLU para aprender representaciones más abstractas de los datos, una capa de convolución unidimensional y capas de SpatialDropout y Dropout que son incluidas para regularizar los datos y reducir el sobreajuste.

Ademas se define un callback ReduceLROnPlateau2 para reducir la tasa de aprendizaje cuando la pérdida en el conjunto de validación deja de mejorar.

In [ ]:
# Max_length se refiere a la secuencia de mayor largo en los corpus, mientras que Mat es la matriz de WE pre-entrenados
def lstm2 (Max_length, X_trainAux, y_trainAux, X_develAux, y_develAux,Mat):
  LR = 0.001
  embedding_layer = tf.keras.layers.Embedding(vocab_size,300,weights=[Mat],
                                          input_length=Max_length,trainable=False)
  sequence_input = Input(shape=(Max_length,), dtype='int32')      # Definimos condiciones de entradas que pasaran a la capa de Embedding
  embedding_sequences = embedding_layer(sequence_input)
  x = SpatialDropout1D(0.2)(embedding_sequences)                  # Regulariza los datos y reduce el sobreajuste.
  x = Conv1D(64, 5, activation='relu')(x)                         # Capa de convolución unidimensional
  x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
  x = Dense(512, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(512, activation='relu')(x)
  outputs = Dense(3, activation='softmax')(x)
  model = tf.keras.Model(sequence_input , outputs)
  model.compile(optimizer=Adam(learning_rate=LR), loss = 'categorical_crossentropy', metrics=[tfa.metrics.F1Score(average='macro',num_classes=3)])
  model.summary()
  ReduceLROnPlateau2 = ReduceLROnPlateau(factor=0.1,min_lr = 0.01, monitor = 'val_loss',verbose = 1)
  training = model.fit(X_trainAux, y_trainAux, batch_size=1024, epochs=5,
                    validation_data=(X_develAux, y_develAux), callbacks=[ReduceLROnPlateau2])
  scores = model.predict(X_develAux, verbose=1, batch_size=10000)
  f1_macro2 = tfa.metrics.F1Score(num_classes=3, average='macro')
  f1_macro2.update_state(y_develAux, scores)
  macro_f1Aux3 = f1_macro2.result().numpy()
  return macro_f1Aux3

In [ ]:
macro_lstm2 = lstm2 (21,X_train_padded,y_trainHot,X_devel_padded,y_develHot,embedding_matrix)
print(macro_lstm2)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 21)]              0         
                                                                 
 embedding_13 (Embedding)    (None, 21, 300)           6531600   
                                                                 
 spatial_dropout1d_12 (Spati  (None, 21, 300)          0         
 alDropout1D)                                                    
                                                                 
 conv1d_16 (Conv1D)          (None, 17, 64)            96064     
                                                                 
 bidirectional_12 (Bidirecti  (None, 128)              66048     
 onal)                                                           
                                                                 
 dense_30 (Dense)            (None, 512)               6604

Vemos que la Macro-F1 obtenida es levemente menor respecto a la del modelo anterior, pero ejecutando este modelo de forma mucho mas rapida y entrenando prácticamente 5 veces mas la cantidad de parámetros respecto al anterior.

--------------------------------------------------

### LSTM con atributos en los Word Embeddings

In [ ]:
#Cargamos los embeddings incluyendo los léxicos:
coleccion_vectores_we_lexicos = cargar_top_embeddings(wordvectors,0,Train_procesado,True)
embedding_model2 = coleccion_vectores_we_lexicos.copy()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_trainP)
vocab_size = len(tokenizer.word_index) + 1

X_train_sequences_lexicos = tokenizer.texts_to_sequences(X_trainP)   # Convertimos los conjuntos en secuencias de índices
X_devel_sequences_lexicos = tokenizer.texts_to_sequences(X_develP)

max_length = max_length_train                                # Ajustamos las secuencias a una longitud máxima, la del Data set de entrenamiento
X_train_padded_lexicos = pad_sequences(X_train_sequences_lexicos, maxlen=max_length)
X_devel_padded_lexicos = pad_sequences(X_devel_sequences_lexicos, maxlen=max_length)

embedding_dim = 302                                          # Creamos una matriz de embedding inicializada con ceros
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():                 # Rellenamos la matriz de embedding con los vectores pre-entrenados
    if word in embedding_model2:
        embedding_matrix[i] = embedding_model2[word]

Las palabras mas comunes del corpus son: ['no', 'pero', 'si', 'mas', 'hoy', 'gracias', 'muy', 'dia', 'ana', 'ue']
Nuestra colección tiene 20030 vectores cargados


In [ ]:
y_trainHot = etiquetas_one_hot(y_train_mod)
y_develHot = etiquetas_one_hot(y_devel)

In [ ]:

def lstm3 (Max_length, X_trainAux, y_trainAux, X_develAux, y_develAux,Mat,nro_epocas):
  model = Sequential()
                                #Esta vez seteamos en 302 la dimension de los vectores en la entrada.
  model.add(Embedding(vocab_size, 302, weights=[Mat],
                      input_length=Max_length, trainable=False))
  model.add(Bidirectional(LSTM(64)))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=[tfa.metrics.F1Score(average='macro',num_classes=3)], run_eagerly=True)
  model.summary()

  for i in range (1,nro_epocas):
    model.fit(X_trainAux, y_trainAux, epochs=1, batch_size=32)
    scores = model.predict(X_develAux, verbose=1, batch_size=10000)
    f1_macro2 = tfa.metrics.F1Score(num_classes=3, average=None)
    f1_macro2.update_state(y_develAux, scores)
    imprimir_resultados((f1_macro2.result().numpy()))
  return f1_macro2

Al entrenar este modelo durante 10 épocas con batch_size = 32, obtuvimos el siguiente resultado:

In [ ]:
macro_lstm3 = lstm3(21,X_train_padded_lexicos,y_trainHot,X_devel_padded_lexicos,y_develHot,embedding_matrix,6)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 21, 302)           6575144   
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              187904    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 3)                 387       
                                                                 
Total params: 6,763,435
Trainable params: 188,291
Non-trainable params: 6,575,144
_________________________________________________________________
1/1 [==============================] - 1s 528ms/step
F1 (P):    0.46777165
F1 (N):    0.63183475
F1 (NONE): 0.4832962
Macro-F1: 0.5276341835657755
1/1 [==============================] - 1s 513ms/step
F1 (P):    0.6086956
F1 (N):    0.65

## Parte 4: Evaluación sobre test

Deben probar los mejores modelos obtenidos en la parte anterior sobre el corpus de test.

También deben comparar sus resultados con un modelo pre-entrenado para análisis de sentimientos de la biblioteca [pysentimiento](https://github.com/pysentimiento/pysentimiento) (deben aplicarlo sobre el corpus de test).



## Evaluación sobre Test
Los dos modelos que brindaron mejores resultados fueron:
- LSTM, en su tercera definicion.
- SVM, utilizando BOWs combinados con TF-IDF, con C = 3, kernel = "rbf" y gamma =2.

#### Prueba con LSTM 3

In [ ]:
# Evaluación sobre test

test_set_2  = test_set.copy()
Test_procesado = procesar_corpus(test_set_2)
X_testP = [''.join(words[1]) for words in Test_procesado]    #Tweets de Test preprocesados
y_test = [label[2] for label in test_set_2]                  #Etiquetas de Test

coleccion_vectores_we_lexicos = cargar_top_embeddings(wordvectors,0,Train_procesado,True)
embedding_model2 = coleccion_vectores_we_lexicos.copy()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_trainP)
vocab_size = len(tokenizer.word_index) + 1

X_test_sequences_lexicos = tokenizer.texts_to_sequences(X_testP)
X_test_padded_lexicos = pad_sequences(X_test_sequences_lexicos, maxlen=max_length)


embedding_matrix_test = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():                 # Rellenamos la matriz de embedding con los vectores pre-entrenados
    if word in embedding_model2:
        embedding_matrix_test[i] = embedding_model2[word]

y_testHot = etiquetas_one_hot(y_test)

Las palabras mas comunes del corpus son: ['no', 'pero', 'si', 'mas', 'hoy', 'gracias', 'muy', 'dia', 'ana', 'ue']
Nuestra colección tiene 20030 vectores cargados


In [ ]:
macro_lstm3 = lstm3(21,X_train_padded_lexicos,y_trainHot,X_test_padded_lexicos,y_testHot,embedding_matrix_test)
print('\nMacro LSTM3 para test: ' + str(macro_lstm3))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 21, 302)           6575144   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              187904    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 6,763,435
Trainable params: 188,291
Non-trainable params: 6,575,144
_________________________________________________________________
Epoch 1/10
575/575 [==============================] - 212s 365ms/step - loss: 0.6167 - f1_score: 0.7313
Epoch 2/10
575/575 [==============================] - 194s 338ms/step - loss: 0.4580 - f1_score: 0.7973
Epoch 3/10
575/575 [========

### Prueba con SVM - BoW

El corpus Test ya fue pre procesado y llevado a los formatos adecuadas en la Parte 2 cuando se realizo el mismo procedimiento para los corpus de Entrenamiento y Desarrollo. Por lo tanto, simplemente hacemos uso de dichas variables globales:

In [ ]:
model_svm_Best = svm.SVC(C=3, kernel= "rbf",gamma= 2)
model_svm_Best.fit(X_train_bowP, y_train_mod)
y_pred_test_BOW_SVM_Best= model_svm_Best.predict(X_test_bowP)
f1_test_BOW_SVM_Best = f1_score(y_test, y_pred_test_BOW_SVM_Best,average=None)
imprimir_resultados(f1_test_BOW_SVM_Best)

F1 (P):    0.6019417475728156
F1 (N):    0.46746575342465757
F1 (NONE): 0.6451612903225805
Macro-F1: 0.5715229304400179


Vemos en las salidas anteriores que las Macro-F1 obtenidas son menores a las que se obtuvieron al evaluar el Corpus de Desarrollo. Esto se puede deber a que el tamaño del Corpus de Test es mayor respecto al de Desarrollo, en aproximadamente 700 tweets. Debido a esto se tienen mas tweets en donde fallar la predicción. Ahora veremos como dichos valores son significativamente menores a los valores de Macro-F1 que se pueden obtener utilizando métodos mas nuevos y eficientes como son las redes neuronales basadas en la arquitectura BERT, entre otras.

### Evaluamos corpus Test con el modelo pre-entrenado de la biblioteca pysentimiento.

In [ ]:
!pip install pysentimiento

Para que funcione correctamente la biblioteca, se requiere la siguiente versión de transformers:

In [ ]:
!pip install transformers==4.28.0

Imports necesarios:

In [ ]:
from pysentimiento import create_analyzer
import transformers

In [ ]:
transformers.logging.set_verbosity(transformers.logging.ERROR)
analyzer = create_analyzer(task="sentiment", lang="es")

def prediccion_pysentimiento(corpus):
  res_pysentimiento = []                 #en res_pysentimiento almacenamos las predicciones realizadas por pysentimiento,
  for elemento in corpus:                #para todo el corpus de test.
    res = analyzer.predict(elemento[1])
    res_pysentimiento.append(res)
  return res_pysentimiento

In [ ]:
def codificar_etiquetas_pysentimiento(y):
  etiquetas = {'POS': 0, 'NEG': 1, 'NEU': 2}
  return np.array([etiquetas[i] for i in y])


def predicciones_a_numerico(y):
  ret = []
  for elem in y:
    ret.append(elem.output)
  return ret

In [ ]:
y_test_numerico = codificar_etiquetas(y_test)          #Implementada en Parte 3

prediccion_pys = prediccion_pysentimiento(test_set)
y_pysentimiento = predicciones_a_numerico(prediccion_pys)
y_pysentimiento_output = codificar_etiquetas_pysentimiento(y_pysentimiento)

In [ ]:
f1_test2 = f1_score(y_test_numerico, y_pysentimiento_output, average="macro")
print("Macro-F1: " + str(f1_test2))

Macro-F1: 0.7041227233971533


Vemos que la Macro-F1 coincide prácticamente con el valor declarado en el siguiente sitio web:
https://huggingface.co/pysentimiento/robertuito-sentiment-analysis

## Preguntas finales

Responda las siguientes preguntas:

1) ¿Qué modelos probaron para la representación de los tweets?

\

Para la representación de tweets probamos los modelos basados en Bag of Words (BOW) y con Word Embeddings (WE). En el caso de BOW, utilizamos el enfoque que combina BOW con Tf-Idf, y el enfoque que selecciona las k-features mas releavantes del corpus de entrenamiento. La combinación con Tf-Idf resulta útil debido al hecho de que esta tiene en cuenta tanto la frecuencia de aparición de una palabra en un tweet como su rareza en el corpus. Esto significa que las palabras menos frecuentes pero más distintivas tienen un mayor peso, lo que ayuda a capturar mejor la semántica y el contexto del tweet para el análisis sentimental. A su vez dicha aplicación de la técnica Idf, las palabras que aparecen en muchos tweets reciben un puntaje más bajo, lo que les resta importancia y ayuda a filtrar el ruido y las palabras menos informativas. Una desventaja de este enfoque es el tamaño de los vectores que se necesitan para representar los tweets. Estos, por las características de los datos, poseen una dimension mayor a 21000.

Por otro lado el enfoque que selecciona las k-features mas relevantes nos parecía una buena opción a analizar, ya que permite reducir el tamaño de la representación de los tweets y a su vez, como se observó y se volvera a comentar mas adelante, permitió para ciertos valores de hiperparámetros obtener mejores resultados de Macro-F1 respecto al enfoque anterior. Los valores de k que consideramos fueron 250, 500 y 1000. Para obtener la representación, primero seleccionabamos a partir de un Data Set de entrenamiento, las k-features mas relevantes. Una vez obtenido dicho vocabulario se obtenia la representacion de los tweets a partir de la funcion de puntuación $chi$, que evaluaba la secuencia de palabras respecto a dicho vocabulario obtenido anteriormente, generando el BOW que representara al tweet. En relación a lo anterior, se experimento tambien agregar dos valores al final de dichos vectores de dimension $k$. Para dicho caso, el valor de la anteúltima coordenada representaria la cantidad de palabras del tweet que estan incluidas en el lexico positivo brindado via EVA. De forma similar, la última coordenda tendria la cantidad de palabras del tweet que estan incluidas en el léxico negativo.

Respecto a las otros posibles enfoques que se daban como opciones, BOW estandar requería vectores del mismo tamaño que los generados por el enfoque Tf-Idf aunque no consideraba importancia de las palabras por lo que vimos mas adecuado el que utiliza Tf-Idf. BOW filtrando stop-words no corresponde debido a que se filtran las stop-words al preprocesar los corpus por lo que ya estaria siendo considerado. Por ultimo, BOW usando lemas requería tiempos enormes de ejecucion para obtener las representaciones por lo que optamos por prescindir del mismo.

En cuanto a Word Embeddings, probamos ambas representaciones sugeridas en la letra, es decir, tomamos tanto el promedio de los vectores de word embeddings de las palabras de cada tweet para representarlo como la concatenación de los word embeddings. Observamos que la primera representación otorga mejores tiempos de procesamiento que representarlos como la concatenación, esto debido a que tomamos un vector de tamaño mucho menor, pero al no tener la concatenación de los vectores, podemos estar perdiendo información semántica al tomar el promedio, por lo que consideramos que la segunda representación es mejor en este aspecto.

Utilizamos un conjunto de vectores predefinidos para representar nuestros word embeddings, pero observamos que esto podía ser ineficiente al procesar los corpus, por lo que tomamos de nuestro conjunto "train" de tweets todas las palabras que se obtienen luego de aplicar el preprocesamiento, obteniendo así
una colección de vectores mucho mas pequeña otpimizando los tiempos de ejecución y representando los tweets casi en su totalidad como lo haríamos con los word embeddings importados.

Finalmente, consideramos añadir una representación de los léxicos que fueron proporcionados por la letra en la representacion de cada tweet, añadiendo dos valores al final de cada vector, cada uno de estos valores representará la cantidad de palabras positivas y negativas respectivamente que tiene el tweet correspondiente, como ya fue mencionado.

\
2) ¿Aplicaron algún tipo de preprocesamiento de los textos?

\\
Se decidió aplicar preprocesamiento tanto al corpus de entrenamiento como al corpus de test, antes de su evaluación. Esto con el fin de eliminar stop words, que consideramos fundamental a la hora de trabajar con BOW, para reducir el tamaño de los vectores generados. A su vez estas no aportan valor al análisis de sentimiento.
Por otro lado, se eliminaron menciones, urls, tíldes,secuencias de tres o mas veces la misma letra repetida, se sustituyeron groserias por la palabra "insulto", perteneciente al conjunto de lemas "negativos". De forma análoga, las risas se sustituyeron por "jajaja", presente en el conjunto de lemas positivos, para que las risas sean uniformes a su vez. Ademas, se eliminaron los numeros y el simbolo "#", dejando el contendio a continuación del mismo.

\\
3) ¿Qué modelos de aprendizaje automático probaron?

\\

Decidimos probar los modelos de aprendizaje automático basado en atributos mencionados en la letra, es decir, Multi Layer Perceptron (MLP) y Support Vector Machines (SVM). Probamos ambos modelos con diferentes representaciones. Para el caso de word embeddings utilizamos la representación en promedio de los vectores de las palabras de cada tweet preprocesado. Para Bag of Words, elegimos los dos últimos enfoques sugeridos como se mencionó anteriormente: seleccionar las features más relevantes con SelectKBest y BOW combinado con TF-IDF utilizando TfidfVectorizer.

\\

A su vez probamos los modelos Naive Bayes y Regresión Logística declarando un clasificador simple para cada uno de ellos, a modo de experimentación. Para  Naive Bayes utilizamos las entradas representadas mediante BOWs con enfoque Tf-Idf, mientras que para Regresión Logística utilizamos tanto esta representación como también mediante word embeddings promedio. Los resultados sugirieron que Naive Bayes no parece ser una buena opción para esta tarea de PLN, probablemte debido a la independencia condicional que este asume. Por otro lado, el modelo Regresión Logística mostró muy buenos resultados, obteniendose una Macro-F1 de 0.57 aproximadamente para BoW y 60.0 para Word Embeddings Promedio.

\\
4) ¿Qué atributos utilizaron para estos modelos?

\\
En el caso de BOW, se utilizaron los enfoques mencionados anteriormente: select k-features y Tf-Idf, para utilizar a las palabras de los tweets del corpus como atributos, representadas por *X_train_bowP*, donde eran almacenados los BOW de cada tweet previamente preprocesado, del corpus de entrenamiento.


En cuanto a otras variables que se utilizaron para entrenar el modelo y realizar la clasificación, teniamos los atributos categóricos almacenados en vectores. El nombre de dichos vectores comenzaba siempre con la letra "y", y fueron transformados a distintos formatos, segun la función de pérdida utilizada en los clasificadores. Entre ellas, One-hot encoding.


Por otro lado, en BOW combinado con select k-features, podriamos considerar que el valor de K era un atributo que permitía definir la variable de entrada, y una vez iniciado el entrenamiento este no se ajustaba. También lo consideramos hiperparámetro porque lo seleccionamos en algunos casos segun una función de busqueda, aunque no nos quedamos solo con ese valor sino que probamos con tres.

Tanto para el modelo SVM como para MLP, se utilizaron WE pre-entrenados para representar a traves tanto del vector promedio como del vector concatenación a los tweets como atributos. Decidimos generar atributos incorporando los léxicos a nuestra representación contando las palabras positivas y negativas de los tweets e inluyendolos como un número en el vector. Finalmente, para la parte de LSTM, implementamos una capa de Embedding utilizando tanto los vectores de las palabras de forma independiente. Para LSTM 1 y 2, utilizamos vectores de largo 300 sin agregar información de los léxicos, mientras que para LSTM 3 añadimos información de los léxicos obteniendo vectores de largo 302 cuyas dos ultimas coordenadas indican si la palabra correspondiente se encuentra o no en alguno de los dos léxicos.


\\
5) ¿Probaron algún enfoque de aprendizaje profundo?

\\

Probamos el modelo LSTM de redes neuronales recurrentes, donde definimos tres redes LSTM distintas.


Antes de definir las redes, obtuvimos una matriz con los Word Embeddings de las palabras que aparecen en el Corpus de entrenamiento. De esta forma aprovechamos WE pre-entrenados en una capa Embedding de las redes LSTM que describiremos a continuación. Tambien llevamos a los conjuntos de datos de entrenamiento y desarrollo al formato adecuado. Dicho formato refiere a representar mediante índices al conjunto de tweets del corpus de entrenamiento modificado y preprocesado. Luego, con dicho resultado obtenido, llevamos todas las secuencias de índices al mismo largo mediante el uso de la función pad_sequences. A dicha función le indicabamos el largo máximo que un tweet ya sea del corpus de entrenamiento o del de desarrollo, podia tener. En caso de secuencias menores a dicho largo, se le agregaban ceros al incio del vector. Dichas secuencias de vectores de mismo largo son las que finalmente reciben como entradas los modelos de LSTM. Luego las secuencias de etiquetas esperadas de los conjuntos fueron codificadas en formato One-Hot Encoding.

La primer red LSTM utilizaba los WE pre-entrenados que fueron cargados en Parte 2 y puestos en una matriz. La primer Red, disponia para entrenar mas de 85.000 parámetros, contando con otros 6.600.000 de parámetros no entrenables aproximadamente, pues quedaron determinados por los WE pre-entrenados.  Utilizamos tambien una Capa Bidireccional LSTM para capturar tanto el contexto pasado como el futuro de las secuencias en dicha red. A su vez hicimos que el modelo utilizara el optimizador Adam, la función de pérdida categorical_crossentropy y se utiliza la métrica F1 Score para evaluar el rendimiento. La misma fue entrenada durante 5 epocas.


En cuanto a la segunda red considerada, volvimos a entrenar durante 5 épocas. Respecto al modelo anterior, pasamos a utilizar capas mas variadas en busca de mejorar la performance. Entre dichas capas, optamos por dos capas densas (Dense) con activación ReLU para aprender representaciones más abstractas de los datos, una capa de convolución unidimensional y capas de SpatialDropout y Dropout que son incluidas para regularizar los datos y reducir el sobreajuste.
Ademas definimos un callback ReduceLROnPlateau2 para reducir la tasa de aprendizaje cuando la pérdida en el conjunto de validación dejaba de mejorar.
\
Para realizar pruebas respecto a representar los WE incluyendo información de los léxicos como fué mencionado en la Respuesta 5, decidimos probar con ambos modelos, es decir, tanto con la red LSTM 1 como con la red LSTM 2, pero observamos que la red LSTM 1 aportaba mejores resultados, por lo que decidimos solo incluir esta red en la notebook, llamándola LSTM 3. Por lo tanto, como representamos la información que nos aportan los léxicos como dos coordenadas más para nuestros vectores de palabras la red LSTM 3 tiene como entrada de su capa de embedding 302 de dimensión de entrada. Observamos que la Macro-F1 era creciente conforme más epocas utilizábamos, por lo que decidimos fijar un número alto de épocas con respecto a las demás (10).


\\

6) ¿Probaron diferentes configuraciones de hiperparámetros?



Los hiperparámetros de los clasificadores son parámetros que no se aprenden directamente del conjunto de datos, pero que afectan el proceso de entrenamiento y la forma en que se construyen los modelos. Con el fin de obtener el mejor rendimiento posible de los clasificadores, utilizamos diversos hiperparametros para dicho fin, ajustandolos segun los valores de Macro-F1 que eran obtenidos.

En el caso del modelo SVM, utilizamos los siguientes hiperparámetros:

1. C (parámetro de regularización): Controla el equilibrio entre lograr un margen más amplio y minimizar el error de clasificación en el conjunto de entrenamiento. Valores altos de C penalizarán más los errores de clasificación, aunque hacen al modelo más complejo.

2. kernel (núcleo): Especifica la función de transformación utilizada para mapear los datos de entrada en un espacio de mayor dimensión, donde se puede realizar una separación lineal. Los kernels que consideramos son:

   - Lineal (linear): No aplica ninguna transformación y utiliza una función lineal para la separación.
   - Sigmoide (sigmoid): Utiliza una función sigmoide para realizar la transformación.
   - Radial Basis Function (rbf): Transforma el espacio de características original en un espacio de características de mayor dimensión, lo que permite una separación no lineal de las clases. Esta mapea los datos a través de una función no lineal que hace que las muestras sean más fácilmente separables por un hiperplano en el espacio transformado.
   - Polinomial (poly): se mapean los datos de entrada a un espacio de características polinomial en el que utilizamos el paramétro "degree" para determinar el grado del polinomio.

  Dichos tipos de kernel son los mas comunes, y en particular el kernel mas utilizado en clasificadores SVM para anlisis de sentimiento es "rbf"

3. gamma : Es un hiperparámetro crítico en el kernel RBF, un hiperparámetro de ajuste que controla la influencia de cada muestra en el modelo.


Como los valores óptimos de dichos hiperparámetros varian según el conjunto de datos, decidimos utilizar técnicas de validación cruzada para encontrar los valores óptimos que brinden un mejor rendimiento. Para ello utilizamos GridSerach que nos permitia probar 6 combinaciones diferentes, involucrando estas a tres valores diferentes de "C" y las dos posibles funciones para kernel "linear" y "sigmoid". El siguiente código aplicaba la mencionada técnica de validación cruzada con k =5:

\\

model_svm = svm.SVC(random_state=1234)


param_dict = {'C': [ 5, 10, 20],
             'kernel': ['linear', 'sigmoid']}


grid_search = GridSearchCV(model_svm, param_dict, scoring='f1_macro')


grid_search.fit(X_train_bowP, y_train)   #X_train_bowP es el data_set de entrenamiento representado con BOW combiando con TF-IDF.


\\
Finalmente la grilla arrojo que los valores que arrojaron mejor valor de Macro-F1 fueron C=5 y kernel = "linear".


Sin embargo, como se explicó en la parte 3, optamos por probar mas valores ya que el resultado obtenido de la validación cruzada no era muy convincente. De este modo obtuvimos mejores valores con kernel = "rbf", para los tres valores de C: 3, 8 y 20. Eso nos llevo a probar con diferentes valores de gamma, pues este actua en conjunto con kernel rbf, a lo que obtuvimos mejores resultados para C = 3 y gamma 0.1 o 2.

\\

En cuanto a MLP utilizamos los hiperparámetros:

1. Número de capas ocultas: El MLP consta de una o más capas ocultas entre la capa de entrada y la capa de salida.


2. Función de activación: La función de activación se aplica a cada neurona en el MLP y permite la introducción de no linealidad en el modelo. Las funciones de activación consideradas fueron la función sigmoide (sigmoid),y la función de activación rectificada lineal (ReLU).

3. Alpha: El hiperparámetro alpha controla la fuerza de la regularización L2. Es decir, actúa sobre los pesos grandes de manera que valores grandes de alpha penalizan más los pesos grandes y reducen la complejidad del modelo, lo que puede ayudar a prevenir el sobreajuste. Por otro lado, un valor más bajo de alpha permite que los pesos tomen valores más grandes y, por lo tanto, puede permitir un modelo más complejo y flexible, pero con mayor riesgo de sobreajuste.

Al igual que con SVM, utilizamos validación cruzada para determinar que valores optimizaban el valor de la Macro-F1 para el conjunto de entrenamiento representado con BOW combiandos con TF-IDF. Los posibles valores para los hiperparámetros mencionados anteriormente eran:

\\
param_dict = {'hidden_layer_sizes': [(1,), (3,), (10,)],
              'activation': ['relu', 'logistic'],
              'alpha': [0.0001, 0.001, 0.01]}

\\
Luego de la prueba, obtuvimos que los mejores valores eran: hidden_layer_sizes = (3,), activation = "relu", alpha = 0.0001, aunque al igual que con SVM probamos otros valores. Sin embargo, los valores sugeridos para los hiperparámetros: hidden_layer_sizes = (3,), activation = "logistic" y alpha = 0.01, coincidieron con los que daban una mejor Macro-F1 para tweets representados con los distintos enfoques de BOW.

Para Word embeddings concatenación no se pudo llevar a cabo esta medida para ninguno de los dos modelos por el tiempo que supone ejecutar, esto se debe a que los word embeddings concatenados que tomamos son vectores de tamaño 3002 al tomar 10 palabras, por lo que tanto para los concatenados probamos distintos valores manualmente. Para Word Embeddings promedio utilizamos Gridsearch en MLP, obteniendo buenos resultados, pero en SVM decidimos además probar valores manualmente.
\\

Finalmete para LSTM utilizamos los hiperparametros:

1. Embedding ("input_dim", "output_dim", "input_length")

2. Bidirectional LSTM

3. Dense

4. Activation ("softmax")

5. Optimizador ("adam")

6. Loss (funcion de perdida: 'categorical_crossentropy')

7. Epochs

La explicación de los mismos fue resumida en la parte anterior.

En cuanto a los valores de dichos hiperparámetros, realizamos diferentes ejecuciones, donde ibamos variando los mismos. Dichas variaciones determinaban el número de parametros a entrenar de la red, para el cual no nos podíamos exceder debido a que la ejecución podia llegar a no ser soportada por la limitación de la RAM del entorno de ejecución. Finalmente, nos quedamos con la configuración de valores que arrojó mejores resultados, para cada red.

\\

7) ¿Qué enfoque (preprocesamiento + representación de tweets + modelo + atributos/parámetros) obtuvo la mejor Macro-F1?

\\


En la siguiente tabla, se indica los mejores resultados obtenidos, para cada representación bajo la medida Macro-F1. Las entradas vacías simbolizan que no se utilizo el clasificador con esa representación, ya sea, por la naturaleza del clasificador, como por ejemplo naive bayes que se basa en recuentos de palabras o la frecuencia de términos siendo mas conveniente la representación BOW; o simplemente por simplicidad.

In [ ]:
from IPython.display import HTML, display

tabla_html = """
<table style="border-collapse: collapse;">
  <tr style="background-color: gray; color: white;">
    <th></th>
    <th style="padding: 8px;">BOW+TF-IDF</th>
    <th style="padding: 8px;">BOW+k-feat.</th>
    <th style="padding: 8px;">BOW+k-f.+2 coord.</th>
    <th style="padding: 8px;">WE(mean vector)</th>
    <th style="padding: 8px;">WE(concat)</th>
  </tr>
  <tr>
    <td style="padding: 8px;">MLP</td>
    <td style="padding: 8px;">0.556</td>
    <td style="padding: 8px;">0.562</td>
    <td style="padding: 8px;">0.562</td>
    <td style="padding: 8px;">0.581</td>
    <td style="padding: 8px;">0.561</td>
  </tr>
  <tr>
    <td style="padding: 8px;">SVM</td>
    <td style="padding: 8px;">0.598</td>
    <td style="padding: 8px;">0.533</td>
    <td style="padding: 8px;">0.549</td>
    <td style="padding: 8px;">0.597</td>
    <td style="padding: 8px;">0.522</td>
  </tr>
  <tr>
    <td style="padding: 8px;">NAIVE BAYES</td>
    <td style="padding: 8px;">0.417</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">-</td>
  </tr>
  <tr>
    <td style="padding: 8px;">REGRESIÓN LOG.</td>
    <td style="padding: 8px;">0.575</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">0.602</td>
    <td style="padding: 8px;">-</td>
  </tr>
  <tr>
    <td style="padding: 8px;">LSTM</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">-</td>
    <td style="padding: 8px;">0.600</td>
    <td style="padding: 8px;">-</td>
  </tr>
</table>
"""

display(HTML(tabla_html))


,BOW+TF-IDF,BOW+k-feat.,BOW+k-f.+2 coord.,WE(mean vector),WE(concat)
MLP,0.556,0.562,0.562,0.581,0.561
SVM,0.598,0.533,0.549,0.597,0.522
NAIVE BAYES,0.417,-,-,-,-
REGRESIÓN LOG.,0.575,-,-,0.602,-
LSTM,-,-,-,0.600,-


El primer enfoque que dió la mejor Macro-F1 fué la Red Neuronal LSTM3, que consiste en aplicar el preprocesamiento habitual para el corpus de Train Procesado, el cual además de agregar los valores de los léxicos como tweets, agrega tweets inventados como 'NONE' en orden, es decir, las categorías del corpus están intercaladas como 'NONE' 'P' 'N' 'NONE' ... obteniendo un corpus equilibrado que observamos que da buenos resultados para las redes LSTM. Respecto a la representación, utilizamos los WE con atributos generados por los léxicos como fué mencionado anteriormente. Respecto a los parámetros, como ya se mencionó, debimos de utilizar 302 de dimensión de entrada en la capa de embedding, los hiperparámetros de mejor resultado para este modelo fueron: 10 épocas de entrenamiento, tamaño de batch de 32, optimizador adam y 64 unidades en la capa LSTM.

El segundo enfoque que obtuvo la mejor Macro-F1 consistía en aplicar el preprocesamiento habitual para los Corpus de entrenamiento y desarrollo. Luego representar los tweets mediante BOWs combinado con Tf-Idf, y codificando el conjunto de etiquetas mediante One-Hot Encoding, para pasarle finalmente dichas variables a un modelo SVM con hiperparámetros de valor C=3, kernel=rbf y gamma=2. Dado todo lo anterior, se obtuvo una Macro-F1 igual a 0.5976970560303894

\\

8) ¿Qué clase es la mejor clasificada por este enfoque? ¿Cuál es la peor? ¿Por qué piensan que sucede esto?

\\
Respecto al enfoque anterior de LSTM (3), traemos los datos obtenidos de la correspondiente seccion donde se realizo la prueba y se obtuvo:

\\

F1 (P):    0.6272618
F1 (N):    0.66490066
F1 (NONE): 0.5088235
Macro-F1: 0.6003286838531494

En un principio, la F1 de None estaba muy alejada de este valor obtenido. Para intentar mejorarla, se introdujeron como ya fue mencioando 3354 "tweets" nuevos para cada clase, mezclados entre ellos. Dicha estrategia mejoró los resultados en todos los modelos en general, donde se aplicó dicho Corpus modificado.

Observamos que la clase mejor clasificada son los Negativos, mientras que la peor clase son los Neutros. Consideramos que esto sucede porque en este enfoque aplicamos tanto la estategia de generar tweets a partir de los léxicos como la de añadir atributos de cantidad de palabras negativas y positivas a los word-embeddings. Ambas acciones potencian las dos clases con ventaja en este método.

\\

9) ¿Cómo son sus resultados en comparación con los de pysentimiento? ¿Por qué piensan que sucede esto?

\\

Los resultados son inferiores respecto a los que se logran mediante pysentimiento. Esto se debe a varias razones. Entre ellas:


1. Preentrenamiento en datos relevantes: Pysentimiento se entrena en un gran conjunto de datos de texto, que incluye una amplia variedad de expresiones y estilos de lenguaje. Dentro de los Data sets que se utilizaron para entrenarlo, se encuentran data sets proprcionados por TASS, el mismo grupo que creó el corpus de entrenamiento que utilizamos en el laboratorio. Cabe destacar que aunque nuestros corpus no los tenían, pysentimiento se entrenó también con tweets que poseen emojis, entre otras características.

2. Pysentimiento está diseñado para capturar y analizar el contexto y la emotividad de los textos, lo que le permite comprender y extraer los sentimientos expresados en los tweets de manera efectiva.

3. Uso de técnicas de procesamiento de lenguaje natural avanzadas: Pysentimiento utiliza técnicas de procesamiento de lenguaje natural avanzadas, como el uso de modelos de lenguaje preentrenados y algoritmos de aprendizaje automático, para comprender y analizar los textos de manera más precisa. Estas técnicas permiten que pysentimiento capture la información semántica y sintáctica relevante de los tweets, lo que contribuye a una mejor detección de sentimientos. Uno de dichos algoritmos de Deep Learning, está basado en Redes neuronales de arquitectura Transformers. Dichas redes representan hoy en día el estado del arte en varias ramas de PLN, entre ellas Análisis de Sentimiento.

4. Adaptación a dominios específicos: Pysentimiento se ha entrenado en una variedad de dominios, lo que le permite adaptarse a diferentes contextos y estilos de texto. Los tweets son un dominio específico con características lingüísticas y emocionales únicas, y el entrenamiento de pysentimiento en un conjunto de datos diverso ayuda a capturar esas características y lograr un rendimiento sólido en el análisis de sentimientos en tweets.

\\
En nuestro caso, nos restringimos a probar distintos algoritmos de aprendizaje automático y de Deep Learning, sin procurar combinar estos ya que el objetivo era experimentar con distintos modelos que se presentaron a lo largo del curso.

Por otro lado, nuestro Corpus de entrenamiento es significativamente menor al utilizado por pysentimiento. El utilizado en este laboratorio contaba con solamente 8314 tweets, aunque como ya fue mencionado, utilizamos los conjuntos de léxicos positivos y negativos para llevarlo a 18373 "tweets". Dicho número no es comparable a los 500 millones de tweets que contaba el corpus de entrenamiento de pysentimiento. Otro punto relacionado a lo anterior, es que estos se encuentran desbalanceados, lo que provoca que en algunos métodos, una de las tres clases tenga un rendimiento destacablemente inferior al resto. Observamos que para los métodos que utilizan el corpus train sin agregar información adicional, la clase 'N' es la que obtiene una peor F1 de las tres categorías, mientras que para los enfoques que utilizan información de los léxicos, esta clase es 'NONE', lo cual tiene mucho sentido ya que en train normal es donde hay menos tweets negativos y al agregar información de léxicos es natural que aumente la precisión de las clases 'P' y 'N'.

Otro factor a tener en cuenta, es que además de hacer uso de modelos mas actuales y potentes, al tratarse de un proyecto de semejante porte, se habrá contado con muchas mas horas de pruebas analíticas y de Ensayo-Error en busca de optimizar modelos que ya de por si seguramente ofrecían resultados mejores a los nuestros.

En relación a lo anterior, para formar la matriz de embeddings que pasamos como parámetro a las redes LSTM, solo pudimos considerar los embeddings correspondientes a las palabras que aparecían en el corpus de desarrollo. Esto debido a que estabamos restringidos por la RAM del ambiente Colab. De otro modo con mas recursos se podría, por ejemplo, pasarle a dichas redes los 2.000.000 de WE que brindaba el archivo .vec con los WE pre-entrenados que utilizamos para la tarea, en lugar de los aproximadamente 21000 WE que optamos por pasarle a la capa Embedding de tal modelo. Incluso, algunos de esos WE, eran secuencias de 300 ceros debido a palabras que el preprocesamiento no pudo eliminar y no existe, o que no fueron incluidas en el archivo .vec.

Para nuestra sorpresa, si bien BoW es un método más clásico que word embeddings o Deep Learning, obtuvo muy buen resultado para SVM obteniendo el segundo lugar
de todos los modelos en cuanto a Macro-F1:

F1 (P):    0.6296296296296297 \\
F1 (N):    0.5 \\
F1 (NONE): 0.6634615384615384 \\

Macro-F1: 0.5976970560303894 \\